In [1]:
import importlib, sys, subprocess

need = {
    "python-slugify": "slugify",
    "openpyxl": "openpyxl",
    "pyarrow": "pyarrow",
    "statsmodels": "statsmodels",
    "pmdarima": "pmdarima"
}

for pip_name, mod_name in need.items():
    try:
        importlib.import_module(mod_name)
        print(f"✓ {pip_name} already installed")
    except ImportError:
        print(f"→ Installing: {pip_name}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "--no-deps", pip_name])

# Version check (info only)
import pandas as pd, pyarrow, openpyxl, statsmodels, pmdarima, slugify
print("\nVERSIONS:",
      "\npandas   :", pd.__version__,
      "\npyarrow  :", pyarrow.__version__,
      "\nopenpyxl :", openpyxl.__version__,
      "\nstatsmodels:", statsmodels.__version__,
      "\npmdarima :", pmdarima.__version__)


→ Installing: python-slugify
✓ openpyxl already installed
✓ pyarrow already installed
→ Installing: statsmodels
→ Installing: pmdarima


ModuleNotFoundError: No module named 'patsy'

In [ ]:
# Step 1 / Cell 2: Libraries and helper functions
from pathlib import Path
import json, re
import numpy as np
import pandas as pd
from slugify import slugify

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

# Kaggle directories
SEARCH_DIRS = [Path("/kaggle/working"), Path("/kaggle/input")]

def find_file_by_keywords(keywords, exts=("xlsx","xls","csv")):
    """
    keywords: substrings like ['scratch']
    return: the first found file path (we choose the one with the shortest name)
    """
    candidates = []
    for base in SEARCH_DIRS:
        if not base.exists(): 
            continue
        for ext in exts:
            for p in base.rglob(f"*.{ext}"):
                name_lower = p.name.lower()
                if all(k.lower() in name_lower for k in keywords):
                    candidates.append(p)
    if not candidates:
        return None
    # pick the 'cleaner' (shorter) name
    candidates = sorted(candidates, key=lambda p: (len(p.name), str(p)))
    return candidates[0]

def normalize_columns(df):
    """
    Simplifies Turkish characters and spaces.
    Eg: 'Yeni Kullanıcı Sayısı' -> 'yeni_kullanici_sayisi'
    """
    mapping = {}
    for c in df.columns:
        # slugify: converts Turkish characters to Latin, turns spaces into '-'
        s = slugify(str(c), separator="_")
        s = re.sub(r"[^a-z0-9_]+", "", s)
        s = re.sub(r"__+", "_", s).strip("_")
        mapping[c] = s
    return df.rename(columns=mapping)

def to_numeric_safe(series: pd.Series):
    """
    Safely converts strings that look numeric to numeric.
    - '1.234,56' -> 1234.56
    - '1,234.56' -> 1234.56
    - '1.234' or '1,234' -> assume 1234 (thousands separator)
    """
    if series.dtype.kind in "biufc":
        return series  # already numeric

    s = series.astype(str).str.strip().replace({"": np.nan})
    # keep only numeric/separator characters and sign/space
    s_norm = s.str.replace(r"[^\d\.,\-+eE ]", "", regex=True)

    def _convert(val):
        if pd.isna(val): 
            return np.nan
        text = str(val).strip()
        if text.count(",") > 0 and text.count(".") > 0:
            # type '1.234,56' -> . thousands, , decimal
            text = text.replace(".", "").replace(",", ".")
        elif "," in text and "." not in text:
            # '1234,56' -> ',' decimal
            # but '1,234' could be thousands; heuristic: if comma is 3 from the end, it's thousands
            parts = text.split(",")
            if len(parts[-1]) in (1,2):  # most likely decimal
                text = text.replace(",", ".")
            else:
                text = text.replace(",", "")
        else:
            # '1.234' or '1234.56' -> either thousands or decimal
            parts = text.split(".")
            if len(parts) > 1 and len(parts[-1]) not in (1,2):
                text = text.replace(".", "")
        try:
            return float(text)
        except:
            return np.nan

    return s_norm.apply(_convert)

def infer_datetime(df):
    """
    Date column inference:
    - a single column like 'date'/'tarih'
    - year + month (+day?) combination
    - year + week combination (assumes Monday as week start)
    returns: (df_copy, name of the 'datetime' column or None, inferred_frequency)
    """
    d = df.copy()
    cols = list(d.columns)

    # candidates
    date_like = [c for c in cols if re.search(r"(date|tarih|week|ay|month|yil|year|hafta)", c)]
    date_col = None
    dt = None

    # priority 1: a single 'date'/'tarih' column
    for key in ["date","tarih"]:
        cands = [c for c in cols if c == key or c.endswith("_"+key)]
        if len(cands)==1:
            date_col = cands[0]
            break

    # otherwise, a single column containing 'date|tarih'?
    if not date_col:
        cands = [c for c in cols if re.search(r"(date|tarih)", c)]
        if len(cands)==1:
            date_col = cands[0]

    def _parse_datecol(col):
        out = pd.to_datetime(d[col], errors="coerce", dayfirst=True, infer_datetime_format=True)
        return out

    if date_col:
        dt = _parse_datecol(date_col)
        if dt.notna().any():
            d["__dt__"] = dt
    else:
        # year + month (+day)
        year_cols  = [c for c in cols if re.search(r"(yil|year)", c)]
        month_cols = [c for c in cols if re.search(r"(ay|month)", c)]
        day_cols   = [c for c in cols if re.search(r"(gun|day)", c)]

        if year_cols and month_cols:
            y = pd.to_numeric(df[year_cols[0]], errors="coerce")
            m = pd.to_numeric(df[month_cols[0]], errors="coerce")
            dday = pd.Series(1, index=df.index)
            if day_cols:
                dday = pd.to_numeric(df[day_cols[0]], errors="coerce").fillna(1)
            d["__dt__"] = pd.to_datetime(dict(year=y, month=m, day=dday), errors="coerce")
        else:
            # year + week
            week_cols = [c for c in cols if re.search(r"(hafta|week)", c)]
            if year_cols and week_cols:
                y = pd.to_numeric(df[year_cols[0]], errors="coerce").astype("Int64")
                w = pd.to_numeric(df[week_cols[0]], errors="coerce").astype("Int64")
                # ISO week assumption: Monday start
                # (week 1 may start at the end of the previous year; simple approach:)
                d["__dt__"] = pd.to_datetime(
                    y.astype(str) + "-W" + w.astype(str) + "-1", errors="coerce", format="%G-W%V-%u"
                )

    freq_guess = None
    if "__dt__" in d.columns and d["__dt__"].notna().sum() >= 3:
        # for frequency inference, sort and take unique values
        s = d["__dt__"].dropna().sort_values().unique()
        if len(s) >= 3:
            try:
                freq_guess = pd.infer_freq(pd.DatetimeIndex(s))
            except:
                freq_guess = None

    return d, ("__dt__" if "__dt__" in d.columns else None), freq_guess

def quick_report(name, df, dt_col=None):
    print(f"\n===== DATA INTAKE SUMMARY :: {name} =====")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    if dt_col and dt_col in df.columns:
        dd = df[dt_col].dropna().sort_values()
        if not dd.empty:
            print("Date range:", dd.iloc[0].date(), "→", dd.iloc[-1].date())
            try:
                print("Unique periods:", dd.nunique())
            except:
                pass
    # missing value report
    na = df.isna().mean().sort_values(ascending=False)
    print("\nNull ratios (top 10):")
    print((na.head(10)*100).round(2).astype(str) + "%")
    # numeric detection
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    print("\nNumeric columns:", num_cols[:20])
    print("Sample rows:\n", df.head(5))

def make_schema(df, name):
    schema = {}
    for c in df.columns:
        dtype = str(df[c].dtype)
        null_ratio = float(df[c].isna().mean())
        schema[c] = {"dtype": dtype, "null_ratio": null_ratio}
    out = {
        "dataset": name,
        "n_rows": int(len(df)),
        "n_cols": int(df.shape[1]),
        "schema": schema,
    }
    return out

# output directory
OUT = Path("/kaggle/working/clean")
OUT.mkdir(parents=True, exist_ok=True)


In [ ]:
# Step 1 / Cell 3: Auto-detect file paths
scratch_path = find_file_by_keywords(["scratch"])
trends_path  = find_file_by_keywords(["google","trend"]) or find_file_by_keywords(["kodlama","ilgi"]) 

print("Scratch file:", scratch_path)
print("Trends  file:", trends_path)

assert scratch_path is not None, "Scratch data not found. Ensure the filename contains 'scratch' or provide the path manually in Cell 3."
assert trends_path  is not None, "Google Trends data not found. Ensure the filename contains 'google' and 'trend' (or provide the path manually in Cell 3)."


In [ ]:
# Step 1 / Cell 4C (Rev2): Read Google Trends with a two-row header, preserve category & term (long form, safe)

import numpy as np
import pandas as pd
from pathlib import Path

def _fill_and_slugify_multicol(multi_cols):
    lvl0 = pd.Series(multi_cols.get_level_values(0), dtype="object")
    lvl1 = pd.Series(multi_cols.get_level_values(1), dtype="object")
    lvl0 = lvl0.replace({None: np.nan, "": np.nan}).ffill()  # forward-fill category headers to the right
    cat = lvl0.apply(lambda x: slugify(str(x), separator="_"))
    sub = lvl1.apply(lambda x: slugify(str(x), separator="_"))
    return pd.MultiIndex.from_arrays([cat, sub], names=["category", "term"])

def read_trends_excel_with_categories(path: Path):
    df = pd.read_excel(path, sheet_name=0, header=[0, 1], engine="openpyxl")
    df = df.dropna(how="all").dropna(how="all", axis=1)
    df.columns = _fill_and_slugify_multicol(df.columns)

    # year & month columns: regardless of category, terms named 'yil'/'ay'
    y_candidates = [c for c in df.columns if c[1] in ("yil","year")]
    m_candidates = [c for c in df.columns if c[1] in ("ay","month")]
    assert y_candidates and m_candidates, "Google Trends: 'year' and 'month' columns not found."
    ycol, mcol = y_candidates[0], m_candidates[0]

    # date
    date = pd.to_datetime(
        dict(
            year=pd.to_numeric(df[ycol], errors="coerce"),
            month=pd.to_numeric(df[mcol], errors="coerce"),
            day=1
        ),
        errors="coerce"
    )

    # metric columns (excluding year/month)
    metric_cols = [c for c in df.columns if c not in (ycol, mcol) and c[1] not in ("yil","year","ay","month")]

    # take only metrics, convert to numeric
    metrics = df.loc[:, metric_cols].copy()
    for c in metric_cols:
        metrics[c] = to_numeric_safe(metrics[c])
    # ensure MultiIndex names
    metrics.columns = pd.MultiIndex.from_tuples(metric_cols, names=["category","term"])

    # long form: stack only metrics, add date at the end
    long = (
        metrics
        .assign(date=date)
        .set_index("date")
        .stack(["category","term"])
        .reset_index()
        .rename(columns={0: "interest"})
    )
    # column order and cleanup
    long = long[["date","category","term","interest"]]
    long = long.dropna(subset=["date"]).sort_values(["category","term","date"]).reset_index(drop=True)
    long.loc[(long["interest"] < 0) | (long["interest"] > 100), "interest"] = np.nan

    return long

trends_long = read_trends_excel_with_categories(trends_path)
quick_report("Google Trends (long, with categories)", trends_long, dt_col="date")

# Save
trends_long.to_parquet(OUT / "google_trends_long.parquet", index=False)
trends_long.to_csv(OUT / "google_trends_long.csv", index=False, encoding="utf-8")
with open(OUT / "google_trends_schema.json", "w", encoding="utf-8") as f:
    json.dump(make_schema(trends_long, "google_trends_long"), f, indent=2, ensure_ascii=False)

print("\nSaved:", OUT / "google_trends_long.parquet")


In [ ]:
# Step 1 / Cell 5C (Rev2): Read Scratch with a two-row header, preserve category & metric (wide + long, safe)

import numpy as np
import pandas as pd
from pathlib import Path

def read_scratch_excel_with_categories_v2(path: Path):
    # Read with multi-row header
    df = pd.read_excel(path, sheet_name=0, header=[0, 1], engine="openpyxl")
    df = df.dropna(how="all").dropna(how="all", axis=1)
    df.columns = _fill_and_slugify_multicol(df.columns)  # names=["category","metric"]

    # year & month columns (terms named 'yil'/'ay')
    y_candidates = [c for c in df.columns if c[1] in ("yil","year")]
    m_candidates = [c for c in df.columns if c[1] in ("ay","month")]
    assert y_candidates and m_candidates, "Scratch: 'year' and 'month' columns not found."
    ycol, mcol = y_candidates[0], m_candidates[0]

    # Date series
    date = pd.to_datetime(
        dict(
            year=pd.to_numeric(df[ycol], errors="coerce"),
            month=pd.to_numeric(df[mcol], errors="coerce"),
            day=1
        ),
        errors="coerce"
    )

    # Metric columns: all pairs except year/month
    metric_cols = [c for c in df.columns if c not in (ycol, mcol) and c[1] not in ("yil","year","ay","month")]

    # Copy only metrics and convert to numeric
    metrics = df.loc[:, metric_cols].copy()
    for c in metric_cols:
        metrics[c] = to_numeric_safe(metrics[c])

    # --- WIDE table (columns with simple names: new_projects, new_users, new_comments) ---
    simple_names = [term for (cat, term) in metric_cols]  # use the lower name since the top category is single
    wide = pd.DataFrame({"date": date})
    for col, new_name in zip(metric_cols, simple_names):
        wide[new_name] = metrics[col].values
    wide = wide.sort_values("date").reset_index(drop=True)

    # --- LONG table (keep category) ---
    # metrics columns are MultiIndex; stack and add 'date'
    metrics.columns = pd.MultiIndex.from_tuples(metric_cols, names=["category","metric"])
    long = (
        metrics.assign(date=date)
               .set_index("date")
               .stack(["category","metric"])
               .reset_index()
               .rename(columns={0: "value"})
               .sort_values(["category","metric","date"])
               .reset_index(drop=True)
    )

    # Frequency inference
    try:
        scratch_freq = pd.infer_freq(pd.DatetimeIndex(wide["date"].dropna().unique()))
    except:
        scratch_freq = None

    return wide, long, "date", scratch_freq

# Run
scratch_df, scratch_long, scratch_dt, scratch_freq = read_scratch_excel_with_categories_v2(scratch_path)

quick_report("Scratch Wide (with categories kept separately)", scratch_df, dt_col=scratch_dt)
print("Inferred frequency (Scratch):", scratch_freq)

# Save (wide + long)
fname_base = "scratch_clean"
scratch_df.to_parquet(OUT / f"{fname_base}.parquet", index=False)
scratch_df.to_csv(OUT / f"{fname_base}.csv", index=False, encoding="utf-8")

scratch_long.to_parquet(OUT / f"{fname_base}_long.parquet", index=False)
scratch_long.to_csv(OUT / f"{fname_base}_long.csv", index=False, encoding="utf-8")

with open(OUT / "scratch_schema.json", "w", encoding="utf-8") as f:
    json.dump(make_schema(scratch_df, "scratch_clean (wide)"), f, indent=2, ensure_ascii=False)

print("\nSaved:", OUT / f"{fname_base}.parquet", "and", OUT / f"{fname_base}_long.parquet")


In [ ]:
# Step 1 / Cell 6C: Categorized summary message

from textwrap import dedent

terms_preview = trends_long.groupby("category")["term"].nunique().reset_index().sort_values("term", ascending=False)
cat_list = trends_long["category"].dropna().unique().tolist()

msg = dedent(f"""

[1] Google Trends (long, with categories) — Summary
- shape: {trends_long.shape[0]} rows x {trends_long.shape[1]} cols
- categories: {cat_list}
- term counts (top 8):
{terms_preview.head(8).to_string(index=False)}
- date range: {trends_long['date'].min().date()} → {trends_long['date'].max().date()}
- sample rows:
{trends_long.head(8).to_string(index=False)[:900]}

[2] Scratch — Summary
- wide shape: {scratch_df.shape[0]} rows x {scratch_df.shape[1]} cols
- long shape: {scratch_long.shape[0]} rows x {scratch_long.shape[1]} cols
- date range: {scratch_df['date'].min().date()} → {scratch_df['date'].max().date()}
- metrics (wide): {[c for c in scratch_df.columns if c!='date']}
- inferred frequency (scratch): {scratch_freq}

[3] Saved files (Kaggle /kaggle/working/clean):
- google_trends_long.parquet / .csv  (columns: date, category, term, interest)
- scratch_clean.parquet / .csv       (wide: date + metrics)
- scratch_clean_long.parquet / .csv  (long: date, category, metric, value)
""").strip()

print(msg)


In [ ]:
# Step 2 / Cell A1: Helper functions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")

def zfill_month(dt: pd.Series):
    """
    Resets months to the 'first day of the month'.
    Why? to_timestamp('MS') is deprecated; we use 'M' + how='start'.
    """
    s = pd.to_datetime(dt, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")


def ma(s: pd.Series, k=3):
    """Simple moving average (default k=3)."""
    return s.rolling(k, min_periods=1, center=False).mean()

def minmax_0_100(s: pd.Series):
    """Min-max normalize to the 0–100 range (for direct comparison)."""
    s = s.astype(float)
    lo, hi = s.min(), s.max()
    if pd.isna(lo) or pd.isna(hi) or hi == lo:
        return pd.Series(np.nan, index=s.index)
    return (s - lo) / (hi - lo) * 100.0

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.dpi"] = 110


In [ ]:
# Step 2 / Cell A2: Load cleaned data
trends_long = pd.read_parquet(CLEAN / "google_trends_long.parquet")
scratch_wide = pd.read_parquet(CLEAN / "scratch_clean.parquet")  # date, new_projects, new_users, new_comments

# Normalize dates (start of month)
trends_long["date"] = zfill_month(trends_long["date"])
scratch_wide["date"] = zfill_month(scratch_wide["date"])

# Short summary
print("Trends:", trends_long.shape, trends_long["date"].min().date(), "→", trends_long["date"].max().date(),
      "| categories:", trends_long["category"].unique().tolist())
print("Scratch:", scratch_wide.shape, scratch_wide["date"].min().date(), "→", scratch_wide["date"].max().date())


In [ ]:
# Step 2 / Cell A3: Summary tables (categories, terms, scratch metrics)

# 1) Number of terms per category
terms_per_cat = trends_long.groupby("category")["term"].nunique().sort_values(ascending=False)
print("Term count / category:\n", terms_per_cat, "\n")

# 2) Average interest (0-100) for each term, top 10
term_means = (trends_long.groupby(["category","term"])["interest"]
              .mean().sort_values(ascending=False))
print("Highest average interest (top 10):\n", term_means.head(10), "\n")

# 3) Scratch summary statistics
scratch_desc = scratch_wide.describe()[["new_projects","new_users","new_comments"]]
print("Scratch metrics — summary statistics:\n", scratch_desc, "\n")

# 4) Intersection date range (for correlation later)
start = max(trends_long["date"].min(), scratch_wide["date"].min())
end   = min(trends_long["date"].max(), scratch_wide["date"].max())
print("Intersection date range:", start.date(), "→", end.date())


In [ ]:
# Step 2 / Cell A4: Category averages (3M MA)

# Average interest per month-category
cat_monthly = (trends_long.groupby(["date","category"])["interest"]
               .mean().reset_index())
pivot_cat = cat_monthly.pivot(index="date", columns="category", values="interest")

# 3-month moving average
pivot_cat_ma = pivot_cat.apply(lambda s: ma(s, 3))

plt.figure()
for c in pivot_cat_ma.columns:
    plt.plot(pivot_cat_ma.index, pivot_cat_ma[c], label=c)
plt.title("Google Trends — Category Average (3M MA)")
plt.xlabel("Date"); plt.ylabel("Interest (0–100)")
plt.legend(loc="upper left", ncol=2, fontsize=9)
plt.tight_layout()
plt.show()


In [ ]:
# Step 2 / Cell A5: Scratch metrics (3M MA)

sw = scratch_wide.set_index("date")[["new_projects","new_users","new_comments"]]
sw_ma = sw.apply(lambda s: ma(s, 3))

plt.figure()
for c in sw_ma.columns:
    plt.plot(sw_ma.index, sw_ma[c], label=c)
plt.title("Scratch — New Projects / Users / Comments (3M MA)")
plt.xlabel("Date"); plt.ylabel("Count")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
# Step 2 / Cell A6: 'platformlar_araclar: scratch' vs 'new_users' (normalize → 0–100)

# 1) From Google Trends, the 'scratch' term (in the platformlar_araclar category)
scratch_term = (trends_long.query("category == 'platformlar_araclar' and term == 'scratch'")
                .set_index("date")["interest"].rename("trend_scratch"))

# 2) Scratch new users
new_users = scratch_wide.set_index("date")["new_users"].rename("scratch_new_users")

# 3) Restrict to the common date range
both = pd.concat([scratch_term, new_users], axis=1).dropna()

# 4) 3M MA + normalize to 0–100
both_ma = both.apply(lambda s: ma(s, 3))
both_norm = both_ma.apply(minmax_0_100)

plt.figure()
plt.plot(both_norm.index, both_norm["trend_scratch"], label="Trend: scratch (0–100, 3M MA)")
plt.plot(both_norm.index, both_norm["scratch_new_users"], label="Scratch new_users (0–100, 3M MA)")
plt.title("Google Trends 'scratch' vs Scratch 'new_users' — normalized comparison")
plt.xlabel("Date"); plt.ylabel("Normalized Score (0–100)")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
# Step 2 / Cell A7: YoY change (annual %)

# Scratch: YoY %
sw = scratch_wide.set_index("date")[["new_projects","new_users","new_comments"]]
scratch_yoy = (sw.pct_change(12) * 100).rename(columns=lambda c: c + "_yoy_pct")

# Trends: category average YoY % (interest is 0–100, used to see direction of change)
cat_y = (trends_long.groupby(["date","category"])["interest"].mean()
         .unstack("category"))
trends_yoy = (cat_y.pct_change(12) * 100).add_suffix("_yoy_pct")

# Intersection and sample table
yoy_merged = scratch_yoy.join(trends_yoy, how="inner")
print("YoY merged table shape:", yoy_merged.shape)
print(yoy_merged.tail(12).round(2).to_string())


In [ ]:
# Step 2 / Cell A8: Save the merged dataset (we’ll use it for correlation and lag analysis)

# Add some selected terms as well (example: coding_for_kids, code_org, blockly)
sel_terms = [
    ("genel_kodlama_ilgisi","coding_for_kids"),
    ("platformlar_araclar","scratch"),
    ("platformlar_araclar","code_org"),
    ("platformlar_araclar","blockly"),
    ("oyun_tabanli_ogrenme","coding_games")
]

# wide table: each selected term becomes a column
dfs = []
for cat, term in sel_terms:
    s = (trends_long.query("category == @cat and term == @term")
         .set_index("date")["interest"].rename(f"{cat}__{term}"))
    dfs.append(s)
trends_sel_wide = pd.concat(dfs, axis=1)

merged_for_next = scratch_wide.set_index("date").join(trends_sel_wide, how="inner").reset_index()
merged_for_next.to_parquet(CLEAN / "merged_step2.parquet", index=False)
merged_for_next.to_csv(CLEAN / "merged_step2.csv", index=False)

print("Saved:", CLEAN / "merged_step2.parquet")
print("Columns:", merged_for_next.columns.tolist())
print("Date range:", merged_for_next['date'].min().date(), "→", merged_for_next['date'].max().date())
print("Sample rows:\n", merged_for_next.head(5))


In [ ]:
# Step 2+ / Cell B1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def cat_pivot(tr_df: pd.DataFrame, cat: str):
    """Convert the selected category to wide form (date x term)."""
    df = (tr_df.query("category == @cat")
                .pivot(index="date", columns="term", values="interest")
                .sort_index())
    df_ma = df.rolling(3, min_periods=1).mean()  # 3M MA
    return df, df_ma

def top_terms(df_ma: pd.DataFrame, k=5, mode="mean"):
    """Top-K terms: 'mean' (entire period) or 'recent' (last 12 months avg.)"""
    if mode == "recent":
        ref = df_ma.tail(12).mean(numeric_only=True)
    else:
        ref = df_ma.mean(numeric_only=True)
    return ref.sort_values(ascending=False).head(k).index.tolist()

def minmax_0_100_cols(df: pd.DataFrame):
    out = df.copy()
    for c in out.columns:
        s = out[c].astype(float)
        lo, hi = s.min(), s.max()
        out[c] = np.where(hi>lo, (s-lo)/(hi-lo)*100.0, np.nan)
    return out


In [ ]:
# Step 2+ / Cell B2
CATS = trends_long["category"].unique().tolist()  # all categories
K = 5  # number of terms to display per category

for CAT in CATS:
    df, df_ma = cat_pivot(trends_long, CAT)
    terms = top_terms(df_ma, k=K, mode="mean")
    plt.figure(figsize=(11,4))
    for t in terms:
        plt.plot(df_ma.index, df_ma[t], label=t)
    plt.title(f"Google Trends — {CAT} | Top-{K} terms (3M MA)")
    plt.xlabel("Date"); plt.ylabel("Interest (0–100)")
    plt.legend(loc="upper left", ncol=2, fontsize=9)
    plt.tight_layout()
    plt.show()


In [ ]:
# Step 2+ / Cell B3
for CAT in CATS:
    df, df_ma = cat_pivot(trends_long, CAT)
    terms = top_terms(df_ma, k=5, mode="recent")  # most popular based on the last 12 months
    df_norm = minmax_0_100_cols(df_ma[terms])

    plt.figure(figsize=(11,4))
    for t in terms:
        plt.plot(df_norm.index, df_norm[t], label=t)
    plt.title(f"{CAT} | Top-5 (last 12 months popular) — normalized 0–100, 3M MA")
    plt.xlabel("Date"); plt.ylabel("Normalized (0–100)")
    plt.legend(loc="upper left", ncol=2, fontsize=9)
    plt.tight_layout()
    plt.show()


In [ ]:
# Step 2+ / Cell B4
for CAT in CATS:
    df, df_ma = cat_pivot(trends_long, CAT)
    # only columns with sufficient data
    df_ma = df_ma.loc[:, df_ma.notna().mean() > 0.7]
    if df_ma.shape[1] < 2:
        continue
    corr = df_ma.corr()

    plt.figure(figsize=(6,5))
    im = plt.imshow(corr, vmin=-1, vmax=1)
    plt.title(f"{CAT} | Term-Term Correlation (3M MA)")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    ticks = np.arange(len(corr.columns))
    plt.xticks(ticks, corr.columns, rotation=60, ha="right", fontsize=8)
    plt.yticks(ticks, corr.index, fontsize=8)
    plt.tight_layout()
    plt.show()


In [ ]:
# Step 2+ / Cell B5
from numpy.polynomial.polynomial import polyfit

def slope_last_n(series: pd.Series, n=12):
    s = series.dropna().tail(n)
    if len(s) < max(6, n//2):
        return np.nan
    # Normalize to 0–100 to make slope comparable
    lo, hi = s.min(), s.max()
    if hi <= lo:  # constant series
        return 0.0
    s = (s - lo) / (hi - lo) * 100.0
    x = np.arange(len(s))
    b1 = polyfit(x, s.values, 1)[1]  # slope
    return b1

for CAT in CATS:
    df, df_ma = cat_pivot(trends_long, CAT)
    slopes = {t: slope_last_n(df_ma[t], 12) for t in df_ma.columns}
    top_up = pd.Series(slopes).sort_values(ascending=False).head(10).round(2)
    print(f"\n=== {CAT} | Last 12 months 'momentum' (slope) — Top 10 increases ===")
    print(top_up.to_string())


In [ ]:
# Step 3 / C1: Load, align, and pivot data to wide form
import pandas as pd, numpy as np
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")

# The cleaned data we saved
trends_long = pd.read_parquet(CLEAN / "google_trends_long.parquet")  # date, category, term, interest
scratch_wide = pd.read_parquet(CLEAN / "scratch_clean.parquet")      # date, new_projects, new_users, new_comments

# Fix dates to the start of the month
def zfill_month(dt: pd.Series):
    s = pd.to_datetime(dt, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

trends_long["date"] = zfill_month(trends_long["date"])
scratch_wide["date"] = zfill_month(scratch_wide["date"])

# Trends → wide: each column 'category__term'
tw = trends_long.pivot(index="date", columns=["category","term"], values="interest").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
# Smooth slightly with a 3-month MA (reduces noise for correlation)
tw_ma = tw.rolling(3, min_periods=1).mean()

# Scratch → set index
sw = scratch_wide.set_index("date")[["new_projects","new_users","new_comments"]].sort_index()
sw_ma = sw.rolling(3, min_periods=1).mean()

# Common range (around 2007-03 → 2024-06)
start = max(tw_ma.index.min(), sw_ma.index.min())
end   = min(tw_ma.index.max(), sw_ma.index.max())
tw_ma = tw_ma.loc[start:end]
sw_ma = sw_ma.loc[start:end]

print("tw_ma shape:", tw_ma.shape, "| sw_ma shape:", sw_ma.shape, "| range:", start.date(), "→", end.date())


In [ ]:
# Step 3 / C2: Helpers (lagged correlation, z-score)

def zscore(s: pd.Series):
    s = s.astype(float)
    return (s - s.mean()) / s.std(ddof=0)

def corr_with_lags(tr_series: pd.Series, sc_series: pd.Series, lags=range(-12,13), method="pearson"):
    """
    tr_series: Trends series (x_t)
    sc_series: Scratch metric (y_t)
    Positive lag = Trends leads (correlation of x_t with y_{t+lag})
                   (in practice we shift the scratch series forward by +lag)
    """
    out = []
    for k in lags:
        sc_shift = sc_series.shift(k)
        valid = tr_series.notna() & sc_shift.notna()
        n = int(valid.sum())
        if n >= 12:
            x = tr_series[valid]
            y = sc_shift[valid]
            if method == "pearson":
                r = x.corr(y)
            else:
                r = x.rank().corr(y.rank())
            out.append({"lag": k, "r": r, "n": n})
    return pd.DataFrame(out)

print("Helpers ready.")


In [ ]:
# Step 3 / C3: Find the best correlations for all terms vs. each Scratch metric
TERMS = tw_ma.columns.tolist()
METRICS = ["new_projects","new_users","new_comments"]
LAGS = range(-12, 13)

rows = []
for term in TERMS:
    x = zscore(tw_ma[term])  # standardize
    for m in METRICS:
        y = zscore(sw_ma[m])
        # Pearson
        dfp = corr_with_lags(x, y, lags=LAGS, method="pearson")
        if not dfp.empty:
            pbest = dfp.iloc[dfp["r"].abs().argmax()]
            rows.append({"term": term, "metric": m, "method":"pearson",
                         "best_lag": int(pbest["lag"]), "r": float(pbest["r"]), "n": int(pbest["n"])})
        # Spearman
        dfs = corr_with_lags(x, y, lags=LAGS, method="spearman")
        if not dfs.empty:
            sbest = dfs.iloc[dfs["r"].abs().argmax()]
            rows.append({"term": term, "metric": m, "method":"spearman",
                         "best_lag": int(sbest["lag"]), "r": float(sbest["r"]), "n": int(sbest["n"])})

corr_summary = pd.DataFrame(rows).sort_values("r", key=lambda s: s.abs(), ascending=False)
corr_summary.head(15)


In [ ]:
# Step 3 / C4: Plot the top 6 matches
import matplotlib.pyplot as plt
def minmax_0_100(s: pd.Series):
    s = s.astype(float)
    lo, hi = s.min(), s.max()
    return (s - lo) / (hi - lo) * 100.0 if hi>lo else s*0+50

TOPK = 6
top_pairs = corr_summary.head(TOPK)

for _, row in top_pairs.iterrows():
    term, metric, lag = row["term"], row["metric"], int(row["best_lag"])
    xt = tw_ma[term]
    yt = sw_ma[metric].shift(lag)  # apply lag in the plot so the curves align

    both = pd.concat([xt.rename("trend"), yt.rename(metric)], axis=1).dropna()
    both = both.apply(minmax_0_100)

    plt.figure(figsize=(10,4))
    plt.plot(both.index, both["trend"], label=f"{term} (Trend)")
    plt.plot(both.index, both[metric], label=f"{metric} (Lag={lag})")
    plt.title(f"Match: {term} ↔ {metric} | Best lag={lag}, r={row['r']:.2f} ({row['method']})")
    plt.xlabel("Date"); plt.ylabel("Normalized (0–100)"); plt.legend(loc="upper left")
    plt.tight_layout(); plt.show()


In [ ]:
# Step 3 / C5: Granger (x causes y?) — Test the top 5 pairs
from statsmodels.tsa.stattools import grangercausalitytests
from pmdarima.arima.utils import ndiffs
import numpy as np

def make_stationary(s: pd.Series):
    s = pd.to_numeric(s, errors="coerce")
    s = np.log1p(s)  # safe for zeros
    d = max(ndiffs(s.dropna(), test='adf', max_d=2), 0)
    s = s.diff(d) if d>0 else s
    return s.dropna(), d

def granger_x_to_y(x: pd.Series, y: pd.Series, maxlag=12):
    df = pd.concat([y.rename("y"), x.rename("x")], axis=1).dropna()
    if len(df) < maxlag*3:
        return None  # data is short
    y_s, dy = make_stationary(df["y"])
    x_s, dx = make_stationary(df["x"])
    df_s = pd.concat([y_s, x_s], axis=1).dropna()
    if len(df_s) < maxlag*3:
        return None
    res = grangercausalitytests(df_s[["y","x"]], maxlag=maxlag, verbose=False)
    out = []
    for lag, r in res.items():
        p = r[0]["ssr_ftest"][1]  # SSR F-test p-value
        out.append({"lag": lag, "p_value": float(p)})
    best = min(out, key=lambda d: d["p_value"])
    return {"best_lag": int(best["lag"]), "best_p": float(best["p_value"]), "all": out, "d_y": int(dy), "d_x": int(dx), "n": int(len(df_s))}

# From the best 10 matches, take "pearson" results and select unique (term, metric)
top_for_granger = (corr_summary.query("method=='pearson'")
                   .drop_duplicates(subset=["term","metric"])
                   .head(10))

grows = []
for _, r in top_for_granger.iterrows():
    term, metric = r["term"], r["metric"]
    x = tw_ma[term]              # trend
    y = sw_ma[metric]            # scratch metric
    g = granger_x_to_y(x, y, maxlag=12)
    if g:
        g["term"] = term; g["metric"] = metric
        grows.append(g)

granger_summary = pd.DataFrame([{
    "term": g["term"], "metric": g["metric"],
    "best_lag": g["best_lag"], "best_p": g["best_p"],
    "d_x": g["d_x"], "d_y": g["d_y"], "n": g["n"]
} for g in grows]).sort_values("best_p")

granger_summary.head(10)


In [ ]:
# Step 3 / C6: Save outputs
corr_summary.to_csv(CLEAN / "corr_lag_summary.csv", index=False)
if 'granger_summary' in globals() and not granger_summary.empty:
    granger_summary.to_csv(CLEAN / "granger_summary.csv", index=False)

print("Saved:")
print("-", CLEAN / "corr_lag_summary.csv")
print("-", CLEAN / "granger_summary.csv")


In [ ]:
# Step 3 / C7: Feedback text
from textwrap import dedent

granger_info = "granger_summary.csv could not be created (insufficient data)" \
               if ('granger_summary' not in globals() or granger_summary.empty) \
               else f"Granger p-values computed for {len(granger_summary)} pairs; lowest p: {granger_summary['best_p'].min():.4f}"

best6 = corr_summary.head(6)[["term","metric","method","best_lag","r","n"]].copy()
best6["r"] = best6["r"].round(3)

msg = dedent(f"""
=== STEP 3 COMPLETED ===

[Summary]
- Pearson/Spearman lag scan: {len(corr_summary)} relationships (all terms × 3 metrics × 2 methods).
- Top 6 strongest relationships:
{best6.to_string(index=False)}
- Granger: {granger_info}

[Files]
- corr_lag_summary.csv
- granger_summary.csv

In [ ]:
# D1: Lag curves (Pearson & Spearman) — scratch vs new_users / new_comments
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")
trends_long = pd.read_parquet(CLEAN / "google_trends_long.parquet")
scratch_wide = pd.read_parquet(CLEAN / "scratch_clean.parquet")

def zfill_month(dt): 
    s = pd.to_datetime(dt, errors="coerce"); 
    return s.dt.to_period("M").dt.to_timestamp(how="start")

trends_long["date"] = zfill_month(trends_long["date"])
scratch_wide["date"] = zfill_month(scratch_wide["date"])

tw = (trends_long.pivot(index="date", columns=["category","term"], values="interest")
                 .sort_index())
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
tw = tw.rolling(3, min_periods=1).mean()

sw = scratch_wide.set_index("date")[["new_users","new_comments"]].rolling(3, min_periods=1).mean()

x = tw["platformlar_araclar__scratch"]
pairs = {"new_users": sw["new_users"], "new_comments": sw["new_comments"]}

def corr_with_lags(x, y, lags=range(-12,13), method="pearson"):
    rows=[]
    for k in lags:
        ys = y.shift(k)
        v = x.notna() & ys.notna()
        if v.sum()>=12:
            r = (x[v].corr(ys[v]) if method=="pearson" else x[v].rank().corr(ys[v].rank()))
            rows.append((k, r))
    return pd.DataFrame(rows, columns=["lag","r"])

for name,y in pairs.items():
    plt.figure(figsize=(7,3.2))
    dfp = corr_with_lags(x,y,method="pearson")
    dfs = corr_with_lags(x,y,method="spearman")
    plt.plot(dfp["lag"], dfp["r"], label="pearson")
    plt.plot(dfs["lag"], dfs["r"], label="spearman", linestyle="--")
    plt.axvline(0,color="k",alpha=.2); plt.axvline(12,color="k",alpha=.1); plt.axvline(-12,color="k",alpha=.1)
    plt.title(f"Lag curve: scratch (trend) ↔ {name}")
    plt.xlabel("Lag (months)  |  +: Trend leads"); plt.ylabel("r")
    plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# D2: Lag=0 correlation after 12-month differencing (seasonality removal)
import numpy as np

def d12(s): 
    return np.log1p(s).diff(12)  # log + 12-month difference

x_d = d12(tw["platformlar_araclar__scratch"])
users_d = d12(sw["new_users"])
comments_d = d12(sw["new_comments"])

for name, y in {"new_users": users_d, "new_comments": comments_d}.items():
    v = x_d.notna() & y.notna()
    if v.sum()>=24:
        print(f"{name} | Pearson r@lag0 (d12):", round(x_d[v].corr(y[v]), 3))
    else:
        print(f"{name} | data is short (d12)")


In [ ]:
# Step 3 / C5-Seasonal: Re-test Granger with seasonal differencing (1..6 months)
import pandas as pd, numpy as np
from pathlib import Path
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

CLEAN = Path("/kaggle/working/clean")

# --- Load data (raw, no MA) ---
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")   # date, category, term, interest
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")        # date, new_projects, new_users, new_comments

def zfill_month(dt):
    s = pd.to_datetime(dt, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

tr["date"] = zfill_month(tr["date"])
sc["date"] = zfill_month(sc["date"])

# Trends -> wide (category__term columns)
tw = tr.pivot(index="date", columns=["category","term"], values="interest").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

# Scratch -> wide
sw = sc.set_index("date")[["new_projects","new_users","new_comments"]].sort_index()

# Common range
start = max(tw.index.min(), sw.index.min())
end   = min(tw.index.max(), sw.index.max())
tw = tw.loc[start:end]
sw = sw.loc[start:end]

# --- Helpers ---
def seasonal_stationary(s: pd.Series):
    """log1p -> diff(12) [+ diff(1) if ADF p>0.05]"""
    s = pd.to_numeric(s, errors="coerce")
    s = np.log1p(s).diff(12).dropna()
    used = "log1p -> diff(12)"
    try:
        p = adfuller(s.dropna(), autolag="AIC")[1]
    except Exception:
        p = np.nan
    if np.isnan(p) or p > 0.05:
        s = s.diff().dropna()
        used += " -> diff(1)"
    return s, used

def granger_test_pair(x: pd.Series, y: pd.Series, maxlag=6):
    df = pd.concat([y.rename("y"), x.rename("x")], axis=1).dropna()
    if len(df) < maxlag * 4:   # do not trust very short series
        return None
    res = grangercausalitytests(df[["y","x"]], maxlag=maxlag, verbose=False)
    rows = []
    for lag, r in res.items():
        p = r[0]["ssr_ftest"][1]  # F-test p-value
        rows.append((lag, p))
    best = min(rows, key=lambda t: t[1])
    return {"best_lag": int(best[0]), "best_p": float(best[1])}

# Candidate pairs: if corr_lag_summary.csv exists, take the top 10 strongest pairs from there; otherwise sensible defaults
try:
    cs = pd.read_csv(CLEAN / "corr_lag_summary.csv")
    cs = cs[cs["method"] == "pearson"].drop_duplicates(subset=["term","metric"])
    pairs = (cs.sort_values("r", key=lambda s: s.abs(), ascending=False)
               .head(10)[["term","metric"]].itertuples(index=False, name=None))
    pairs = list(pairs)
except FileNotFoundError:
    pairs = [
        ("platformlar_araclar__scratch","new_users"),
        ("platformlar_araclar__scratch","new_projects"),
        ("platformlar_araclar__scratch","new_comments"),
        ("egitsel_baglam__stem_for_kids","new_users"),
        ("egitsel_baglam__stem_for_kids","new_projects")
    ]

# --- Test ---
out = []
for term, metric in pairs:
    if term not in tw.columns or metric not in sw.columns:
        continue
    x_raw = tw[term]
    y_raw = sw[metric]
    x_s, x_tr = seasonal_stationary(x_raw)
    y_s, y_tr = seasonal_stationary(y_raw)

    df = pd.concat([y_s.rename("y"), x_s.rename("x")], axis=1).dropna()
    if len(df) < 36:
        out.append({"term": term, "metric": metric, "n": len(df), "best_lag": None, "best_p": None,
                    "sig(p<0.05)": False, "x_transform": x_tr, "y_transform": y_tr, "note": "short series"})
        continue

    g = granger_test_pair(df["x"], df["y"], maxlag=6)
    if g:
        out.append({"term": term, "metric": metric, "n": len(df),
                    "best_lag": g["best_lag"], "best_p": g["best_p"],
                    "sig(p<0.05)": g["best_p"] < 0.05,
                    "x_transform": x_tr, "y_transform": y_tr})
    else:
        out.append({"term": term, "metric": metric, "n": len(df), "best_lag": None, "best_p": None,
                    "sig(p<0.05)": False, "x_transform": x_tr, "y_transform": y_tr})

granger_seasonal = pd.DataFrame(out).sort_values("best_p", na_position="last")
print("=== Granger (with seasonal differencing) — summary ===")
if not granger_seasonal.empty:
    print(granger_seasonal.head(10).to_string(index=False))
    outpath = CLEAN / "granger_seasonal_summary.csv"
    granger_seasonal.to_csv(outpath, index=False)
    print("\nSaved:", outpath)
else:
    print("No suitable result could be produced (data may be insufficient).")


In [ ]:
# Step 4 / E1: Data and split
import pandas as pd, numpy as np
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")
trends_long = pd.read_parquet(CLEAN / "google_trends_long.parquet")   # date, category, term, interest
scratch_wide = pd.read_parquet(CLEAN / "scratch_clean.parquet")       # date, new_projects, new_users, new_comments
corr_sum = pd.read_csv(CLEAN / "corr_lag_summary.csv")                # from Step 3
granger_seas = pd.read_csv(CLEAN / "granger_seasonal_summary.csv")    # if available

def mstart(s): 
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

trends_long["date"] = mstart(trends_long["date"])
scratch_wide["date"] = mstart(scratch_wide["date"])

# Target METRIC: you can switch to 'new_projects' or 'new_comments' if you like.
TARGET = "new_users"

y = scratch_wide.set_index("date")[TARGET].asfreq("MS")
# Train/Test: last 24 months as test
TEST_H = 24
y_train, y_test = y.iloc[:-TEST_H], y.iloc[-TEST_H:]

print("Target:", TARGET)
print("Train range:", y_train.index.min().date(), "→", y_train.index.max().date(), "| n=", len(y_train))
print("Test  range:", y_test.index.min().date(), "→", y_test.index.max().date(), "| n=", len(y_test))


In [ ]:
# Step 4 / E2: Exog candidates
# 1) From 'pearson' results (lag 0..6), the top 3 terms strongest with TARGET
c = (corr_sum.query("method=='pearson' and metric==@TARGET")
             .query("best_lag>=0 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False))
top_terms = c.drop_duplicates("term").head(3)[["term","best_lag"]].values.tolist()

# 2) Also include terms that were significant in Seasonal Granger (p<0.05)
if not granger_seas.empty:
    gsel = granger_seas.query("metric==@TARGET and `sig(p<0.05)`==True")[["term","best_lag"]].values.tolist()
else:
    gsel = []

# 3) Safe fallback: scratch term lag0
fallback = [("platformlar_araclar__scratch", 0)]
candidates = (top_terms + gsel + fallback)

# Deduplicate
seen = set(); exog_terms = []
for t, lag in candidates:
    if t not in seen and t in trends_long.assign(col=trends_long["category"]+"__"+trends_long["term"])["col"].unique():
        exog_terms.append((t, int(lag)))
        seen.add(t)

print("Exog candidates (term, lag):", exog_terms[:6])

# Build the exog matrix
tw = trends_long.pivot(index="date", columns=["category","term"], values="interest")
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
tw = tw.asfreq("MS")

def build_exog(exog_specs, ma=3):
    X = pd.DataFrame(index=tw.index)
    for term, lag in exog_specs:
        s = tw[term].rolling(ma, min_periods=1).mean().shift(lag)  # small MA + specified lag
        X[f"{term}__lag{lag}"] = s
    return X

X_all = build_exog(exog_terms)
X_train, X_test = X_all.loc[y_train.index], X_all.loc[y_test.index]
print("Exog shape:", X_all.shape, "| Train/Test:", X_train.shape, X_test.shape)
print("First columns:", X_all.columns[:6].tolist())


In [ ]:
# Step 4 / E3: Baseline SARIMA specification (auto_arima with y only)
from pmdarima import auto_arima

# silence warnings
import warnings; warnings.filterwarnings("ignore")

sarima_model = auto_arima(y_train, seasonal=True, m=12,
                          information_criterion="aic",
                          stepwise=True, trace=False,
                          suppress_warnings=True,
                          error_action="ignore")
order = sarima_model.order
seasonal_order = sarima_model.seasonal_order

print("Selected SARIMA order:", order, seasonal_order)


In [ ]:
# Step 4 / E4: Baseline SARIMA fit & forecast
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np

base = SARIMAX(y_train, order=order, seasonal_order=seasonal_order,
               enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
base_fc = base.get_forecast(steps=TEST_H).predicted_mean
base_fc.index = y_test.index

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    mae = e.abs().mean()
    rmse = np.sqrt((e**2).mean())
    mape = (e.abs() / y_true.replace(0, np.nan)).mean() * 100
    return {"MAE": float(mae), "RMSE": float(rmse), "MAPE%": float(mape)}

base_metrics = metrics(y_test, base_fc)
print("Baseline (SARIMA) metrics:", base_metrics)


In [ ]:
# Step 4 / E5: SARIMAX variants (individually and all together)

from itertools import combinations

results = []
# Each exog individually
for i in range(min(3, len(X_train.columns))):
    name = X_train.columns[i]
    try:
        model = SARIMAX(y_train, exog=X_train[[name]], order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        fc = model.get_forecast(steps=TEST_H, exog=X_test[[name]]).predicted_mean
        r = metrics(y_test, fc)
        r.update({"model":"SARIMAX", "exog":[name]})
        results.append(r)
    except Exception as e:
        results.append({"model":"SARIMAX","exog":[name],"error":str(e)})

# All exogs together (if many columns, take the first 3)
exog_cols = X_train.columns[:min(3, len(X_train.columns))]
if len(exog_cols) >= 2:
    try:
        model = SARIMAX(y_train, exog=X_train[exog_cols], order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        fc = model.get_forecast(steps=TEST_H, exog=X_test[exog_cols]).predicted_mean
        r = metrics(y_test, fc)
        r.update({"model":"SARIMAX","exog":list(exog_cols)})
        results.append(r)
    except Exception as e:
        results.append({"model":"SARIMAX","exog":list(exog_cols),"error":str(e)})

res = pd.DataFrame(results).sort_values("RMSE", na_position="last")
print("SARIMAX results (best first):\n", res.head(6).to_string(index=False))
best = res.iloc[0] if not res.empty else None


In [ ]:
# Step 4 / E6: Plot and save outputs
import matplotlib.pyplot as plt

out_tbl = pd.DataFrame({"actual": y_test})
out_tbl["baseline"] = base_fc

if best is not None and "error" not in best:
    best_ex = best["exog"]
    ex_cols = best_ex if isinstance(best_ex, list) else [best_ex]
    best_model = SARIMAX(y_train, exog=X_train[ex_cols], order=order, seasonal_order=seasonal_order,
                         enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    best_fc = best_model.get_forecast(steps=TEST_H, exog=X_test[ex_cols]).predicted_mean
    best_fc.index = y_test.index
    out_tbl["exog_best"] = best_fc
else:
    out_tbl["exog_best"] = np.nan

# Plot
plt.figure(figsize=(10,4))
plt.plot(y.index, y, label="actual (train+test)")
plt.plot(out_tbl.index, out_tbl["baseline"], label="baseline (SARIMA)")
if out_tbl["exog_best"].notna().any():
    plt.plot(out_tbl.index, out_tbl["exog_best"], label="exog_best (SARIMAX)")
plt.title(f"Forecast — {TARGET} (last {TEST_H} months test)")
plt.xlabel("Date"); plt.ylabel(TARGET); plt.legend(loc="upper left")
plt.tight_layout(); plt.show()

# Save
out_tbl.to_csv(CLEAN / f"forecast_{TARGET}.csv")
print("Saved:", CLEAN / f"forecast_{TARGET}.csv")
print("Baseline:", base_metrics)
if out_tbl["exog_best"].notna().any():
    print("Exog_best:", metrics(y_test, out_tbl["exog_best"]))


In [ ]:
# Step 4 / E8: Baseline vs SARIMAX comparison summary
import pandas as pd, numpy as np
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

CLEAN = Path("/kaggle/working/clean")
TARGET = "new_users"  # If you selected a different target in E1, it should be the same here as well

def mstart(s): 
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    mae = e.abs().mean()
    rmse = np.sqrt((e**2).mean())
    mape = (e.abs() / y_true.replace(0, np.nan)).mean() * 100
    return {"MAE": float(mae), "RMSE": float(rmse), "MAPE%": float(mape)}

# --- load data ---
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")   # date, category, term, interest
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")        # date, new_projects/new_users/new_comments
tr["date"] = mstart(tr["date"]); sc["date"] = mstart(sc["date"])

y = sc.set_index("date")[TARGET].asfreq("MS")
TEST_H = 24
y_train, y_test = y.iloc[:-TEST_H], y.iloc[-TEST_H:]

# read from baseline forecast file (E6)
fc_path = CLEAN / f"forecast_{TARGET}.csv"
out_tbl = pd.read_csv(fc_path, index_col=0, parse_dates=True)
base_metrics = metrics(y_test, out_tbl["baseline"])
has_sarimax = out_tbl["exog_best"].notna().any()
sarimax_metrics = metrics(y_test, out_tbl["exog_best"]) if has_sarimax else None

# --- rebuild exog candidates (summary of E2) ---
corr_sum = pd.read_csv(CLEAN / "corr_lag_summary.csv")
try:
    gseas = pd.read_csv(CLEAN / "granger_seasonal_summary.csv")
except FileNotFoundError:
    gseas = pd.DataFrame(columns=["term","metric","best_lag","sig(p<0.05)"])

tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

c = (corr_sum.query("method=='pearson' and metric==@TARGET")
             .query("best_lag>=0 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False))
top_terms = c.drop_duplicates("term").head(3)[["term","best_lag"]].values.tolist()
gsel = gseas.query("metric==@TARGET and `sig(p<0.05)`==True")[["term","best_lag"]].values.tolist() if not gseas.empty else []
candidates = top_terms + gsel + [("platforms_tools__scratch", 0)]

seen=set(); exog_specs=[]
for t,lag in candidates:
    if t in tw.columns and t not in seen:
        exog_specs.append((t,int(lag))); seen.add(t)

def build_exog(exog_specs, ma=3):
    X = pd.DataFrame(index=tw.index)
    for term, lag in exog_specs:
        X[f"{term}__lag{lag}"] = tw[term].rolling(ma, min_periods=1).mean().shift(lag)
    return X

X_all = build_exog(exog_specs)
X_train, X_test = X_all.loc[y_train.index], X_all.loc[y_test.index]

# If E5 results are not in memory, retry briefly
try:
    res
    best
except NameError:
    # Find SARIMA specification again
    sarima_model = auto_arima(y_train, seasonal=True, m=12, information_criterion="aic",
                              stepwise=True, trace=False, suppress_warnings=True, error_action="ignore")
    order, seasonal_order = sarima_model.order, sarima_model.seasonal_order
    # baseline
    base_fc = SARIMAX(y_train, order=order, seasonal_order=seasonal_order,
                      enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)\
                      .get_forecast(steps=TEST_H).predicted_mean
    # single exog and all together
    results=[]
    cols = X_train.columns[:min(3, len(X_train.columns))]
    for name in cols:
        try:
            m = SARIMAX(y_train, exog=X_train[[name]], order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
            fc = m.get_forecast(steps=TEST_H, exog=X_test[[name]]).predicted_mean
            r = metrics(y_test, fc); r.update({"model":"SARIMAX","exog":[name]}); results.append(r)
        except Exception as e:
            results.append({"model":"SARIMAX","exog":[name],"error":str(e)})
    if len(cols) >= 2:
        try:
            m = SARIMAX(y_train, exog=X_train[cols], order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
            fc = m.get_forecast(steps=TEST_H, exog=X_test[cols]).predicted_mean
            r = metrics(y_test, fc); r.update({"model":"SARIMAX","exog":list(cols)}); results.append(r)
        except Exception as e:
            results.append({"model":"SARIMAX","exog":list(cols),"error":str(e)})
    res = pd.DataFrame(results).sort_values("RMSE", na_position="last")
    best = res.iloc[0] if not res.empty else None

# print summary
def fmt(m): 
    return f"MAE={m['MAE']:.0f} | RMSE={m['RMSE']:.0f} | MAPE={m['MAPE%']:.2f}%"

print("\n=== EVALUATION SUMMARY ===")
print("Target:", TARGET)
print("Baseline:", fmt(base_metrics))
if has_sarimax:
    print("From file — SARIMAX(exog_best):", fmt(sarimax_metrics))
else:
    print("From file — SARIMAX(exog_best): not available (E6 exog_best empty)")

if best is not None and "error" not in best:
    print("\nRe-run check — Best SARIMAX exog:", best["exog"])
    print("Best SARIMAX metrics:", f"MAE={best['MAE']:.0f} | RMSE={best['RMSE']:.0f} | MAPE={best['MAPE%']:.2f}%")
    # improvement
    def improv(b, s): return {k: (b[k]-s[k])/b[k]*100 for k in ["MAE","RMSE","MAPE%"]}
    imp = improv(base_metrics, {"MAE":best["MAE"],"RMSE":best["RMSE"],"MAPE%":best["MAPE%"]})
    print("Improvement (−% better):", {k: f"{v:.1f}%" for k,v in imp.items()})
else:
    print("\nRe-run check — Best SARIMAX could not be calculated or an error occurred.")

print("\nFirst 6 variants (if any):")
try:
    print(res.head(6).to_string(index=False))
except:
    print("res table not available.")


In [ ]:
# F1: sNaive baseline (y[t-12]) ile kıyas
import pandas as pd, numpy as np
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")
TARGET = "new_users"  # başka hedef deniyorsan bunu değiştir

y = pd.read_parquet(CLEAN / "scratch_clean.parquet").set_index("date")[TARGET].asfreq("MS")
TEST_H = 24
y_train, y_test = y.iloc[:-TEST_H], y.iloc[-TEST_H:]

# sNaive: test dönemi için geçen yılın aynı ayı
snaive_fc = y.shift(12).iloc[-TEST_H:]
assert len(snaive_fc) == len(y_test)

# E6 çıktısından baseline ve varsa en iyi SARIMAX
out_tbl = pd.read_csv(CLEAN / f"forecast_{TARGET}.csv", index_col=0, parse_dates=True)
base_fc = out_tbl["baseline"]
exog_fc = out_tbl["exog_best"] if out_tbl["exog_best"].notna().any() else None

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {
        "MAE": float(e.abs().mean()),
        "RMSE": float(np.sqrt((e**2).mean())),
        "MAPE%": float((e.abs()/y_true.replace(0,np.nan)).mean()*100)
    }

print("=== F1: sNaive vs SARIMA vs SARIMAX ===")
print("sNaive:", metrics(y_test, snaive_fc))
print("SARIMA:", metrics(y_test, base_fc))
if exog_fc is not None:
    print("SARIMAX(exog_best):", metrics(y_test, exog_fc))
else:
    print("SARIMAX(exog_best): yok")


In [ ]:
# F2: En iyi SARIMAX için artık tanı (FIX: to_timestamp -> how="start")
import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf

CLEAN = Path("/kaggle/working/clean")
TARGET = "new_users"

def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

# veriler ve exog'lar
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])
y = sc.set_index("date")[TARGET].asfreq("MS")

tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

# E8'deki seçim mantığı (ilk 3 güçlü aday, lag 0..6)
corr = pd.read_csv(CLEAN / "corr_lag_summary.csv")
cand = (corr.query("method=='pearson' and metric==@TARGET")
             .query("best_lag>=0 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False)
             .drop_duplicates("term").head(3))[["term","best_lag"]].values.tolist()

def build_exog(specs, ma=3):
    X = pd.DataFrame(index=tw.index)
    for t,lag in specs:
        X[f"{t}__lag{lag}"] = tw[t].rolling(ma, min_periods=1).mean().shift(int(lag))
    return X

X = build_exog(cand)
TEST_H=24
y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]
X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]

# SARIMA düzeni
a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
               stepwise=True, suppress_warnings=True)
order, seasonal_order = a.order, a.seasonal_order

m = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=seasonal_order,
            enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)

# standardized residuals
resid = pd.Series(m.filter_results.standardized_forecasts_error[0], index=y_tr.index).dropna()

print("=== F2: Ljung–Box p-değerleri ===")
print(acorr_ljungbox(resid, lags=[12,24], return_df=True))

plt.figure(figsize=(7,3))
plot_acf(resid, lags=24)
plt.title("Artık ACF (SARIMAX)"); plt.tight_layout(); plt.show()


In [ ]:
# F3: Rolling-origin backtest — SARIMA vs SARIMAX (FIX: mstart)
import pandas as pd, numpy as np
from pathlib import Path
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

CLEAN = Path("/kaggle/working/clean")
TARGET = "new_users"

def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])
y = sc.set_index("date")[TARGET].asfreq("MS")
tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

corr = pd.read_csv(CLEAN / "corr_lag_summary.csv")
cand = (corr.query("method=='pearson' and metric==@TARGET")
             .query("best_lag>=0 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False)
             .drop_duplicates("term").head(3))[["term","best_lag"]].values.tolist()

def build_exog(specs, ma=3):
    X = pd.DataFrame(index=tw.index)
    for t,lag in specs:
        X[f"{t}__lag{lag}"] = tw[t].rolling(ma, min_periods=1).mean().shift(int(lag))
    return X

X = build_exog(cand)

def rmse(a,b): 
    e=(a-b).astype(float); 
    return float(np.sqrt((e**2).mean()))

folds=5; horizon=12
r_sarima=[]; r_sarimax=[]
for k in range(folds,0,-1):
    split_end = -k*horizon
    y_tr = y.iloc[:split_end] if split_end!=0 else y
    y_te = y.iloc[split_end: split_end + horizon]
    if len(y_te)<horizon or len(y_tr)<60: 
        continue
    X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]

    a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
                   stepwise=True, suppress_warnings=True)
    order, so = a.order, a.seasonal_order

    sarima = SARIMAX(y_tr, order=order, seasonal_order=so,
                     enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    fc0 = sarima.get_forecast(steps=horizon).predicted_mean

    sarimax = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
                      enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    fc1 = sarimax.get_forecast(steps=horizon, exog=X_te).predicted_mean

    r_sarima.append(rmse(y_te, fc0)); r_sarimax.append(rmse(y_te, fc1))

print("=== F3: Rolling-origin RMSE (12-aylık test, 5 kat) ===")
print("SARIMA  RMSE ort:", round(np.mean(r_sarima),1), " | katlar:", [round(x,1) for x in r_sarima])
print("SARIMAX RMSE ort:", round(np.mean(r_sarimax),1), "| katlar:", [round(x,1) for x in r_sarimax])
print("İyileşme (ort):", round((np.mean(r_sarima)-np.mean(r_sarimax))/np.mean(r_sarima)*100,1), "%")




In [ ]:
# F4: Lag0'ı çıkar, yalnız lag>=1 exog'larla test (FIX: mstart)
import pandas as pd, numpy as np
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

CLEAN = Path("/kaggle/working/clean")
TARGET = "new_users"

def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])

y = sc.set_index("date")[TARGET].asfreq("MS")
corr = pd.read_csv(CLEAN / "corr_lag_summary.csv")

cand = (corr.query("method=='pearson' and metric==@TARGET")
             .query("best_lag>=1 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False)
             .drop_duplicates("term").head(3))[["term","best_lag"]].values.tolist()

tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
X = pd.DataFrame(index=tw.index)
for t,lag in cand:
    X[f"{t}__lag{lag}"] = tw[t].rolling(3, min_periods=1).mean().shift(int(lag))

TEST_H=24
y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]
X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]

a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
               stepwise=True, suppress_warnings=True)
order, so = a.order, a.seasonal_order

m = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
            enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
fc = m.get_forecast(steps=TEST_H, exog=X_te).predicted_mean

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {"MAE": float(e.abs().mean()),
            "RMSE": float(np.sqrt((e**2).mean())),
            "MAPE%": float((e.abs()/y_true.replace(0,np.nan)).mean()*100)}

print("=== F4: Lag≥1-only SARIMAX ===")
print(metrics(y_te, fc))
print("Kullanılan exog:", list(X.columns))


In [ ]:
# G1: Platform terimleri ↔ Scratch metrikleri (lag=0 ve en iyi lag 0..6) — Pearson & Spearman
import pandas as pd, numpy as np
from pathlib import Path

CLEAN = Path("/kaggle/working/clean")
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")

def mstart(s): 
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

tr["date"] = mstart(tr["date"]); sc["date"] = mstart(sc["date"])
tw = tr.pivot(index="date", columns=["category","term"], values="interest").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
tw_ma = tw.rolling(3, min_periods=1).mean()

sw = sc.set_index("date")[["new_projects","new_users","new_comments"]].sort_index()
sw_ma = sw.rolling(3, min_periods=1).mean()

platform_terms = [
    "platformlar_araclar__scratch","platformlar_araclar__tynker","platformlar_araclar__code_org",
    "platformlar_araclar__blockly","platformlar_araclar__kodable","platformlar_araclar__bee_bot",
    "platformlar_araclar__code_a_pillar","platformlar_araclar__matatalab"
]
platform_terms = [t for t in platform_terms if t in tw_ma.columns]

def best_lag_corr(x, y, lags=range(0,7), method="pearson"):
    rows=[]
    for k in lags:
        ys = y.shift(k)
        v = x.notna() & ys.notna()
        if v.sum()>=12:
            r = (x[v].corr(ys[v]) if method=="pearson" else x[v].rank().corr(ys[v].rank()))
            rows.append((k, r, int(v.sum())))
    if not rows: 
        return None
    best = max(rows, key=lambda t: abs(t[1]))
    return {"lag":best[0], "r":best[1], "n":best[2]}

out=[]
for term in platform_terms:
    x = tw_ma[term]
    for m in sw_ma.columns:
        y = sw_ma[m]
        # lag 0
        v = x.notna() & y.notna()
        r0_p = x[v].corr(y[v]); r0_s = x[v].rank().corr(y[v].rank()); n0=int(v.sum())
        # en iyi 0..6
        bp = best_lag_corr(x,y,method="pearson"); bs = best_lag_corr(x,y,method="spearman")
        out.append({
            "term": term.split("__",1)[1],
            "metric": m,
            "r0_pearson": round(r0_p,3), "r0_spearman": round(r0_s,3), "n0": n0,
            "best_lag_p": (None if not bp else bp["lag"]),
            "best_r_p":   (None if not bp else round(bp["r"],3)),
            "best_lag_s": (None if not bs else bs["lag"]),
            "best_r_s":   (None if not bs else round(bs["r"],3)),
            "n_best":     (None if not bp else bp["n"])
        })
platform_compare = pd.DataFrame(out).sort_values(["metric","best_r_s"], ascending=[True,False])
print("=== PLATFORM KARŞILAŞTIRMA (lag=0 ve en iyi 0..6) ===")
print(platform_compare.to_string(index=False))


In [ ]:
# G2: new_users, new_projects, new_comments için sNaive / SARIMA / SARIMAX kıyas tablosu
import pandas as pd, numpy as np
from pathlib import Path
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

CLEAN = Path("/kaggle/working/clean")
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
corr = pd.read_csv(CLEAN / "corr_lag_summary.csv")

def mstart(s): 
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

tr["date"] = mstart(tr["date"]); sc["date"] = mstart(sc["date"])

tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

def build_exog_for_target(target, k=3):
    c = (corr.query("method=='pearson' and metric==@target")
                 .query("best_lag>=0 and best_lag<=6")
                 .sort_values("r", key=lambda s: s.abs(), ascending=False)
                 .drop_duplicates("term").head(k))[["term","best_lag"]].values.tolist()
    X = pd.DataFrame(index=tw.index)
    for t,lag in c:
        X[f"{t}__lag{lag}"] = tw[t].rolling(3, min_periods=1).mean().shift(int(lag))
    return X, [f"{t}__lag{lag}" for t,lag in c]

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {"MAE": float(e.abs().mean()),
            "RMSE": float(np.sqrt((e**2).mean())),
            "MAPE%": float((e.abs()/y_true.replace(0,np.nan)).mean()*100)}

rows=[]
for TARGET in ["new_users","new_projects","new_comments"]:
    y = sc.set_index("date")[TARGET].asfreq("MS")
    TEST_H=24
    y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]
    # sNaive
    snaive_fc = y.shift(12).iloc[-TEST_H:]
    m_snaive = metrics(y_te, snaive_fc)
    # SARIMA düzeni
    a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
                   stepwise=True, suppress_warnings=True)
    order, so = a.order, a.seasonal_order
    sarima = SARIMAX(y_tr, order=order, seasonal_order=so,
                     enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    base_fc = sarima.get_forecast(steps=TEST_H).predicted_mean
    m_sarima = metrics(y_te, base_fc)
    # SARIMAX (ilk 3 exog)
    X, used = build_exog_for_target(TARGET, k=3)
    X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]
    try:
        sarimax = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
                          enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        ex_fc = sarimax.get_forecast(steps=TEST_H, exog=X_te).predicted_mean
        m_sarimax = metrics(y_te, ex_fc)
    except Exception as e:
        m_sarimax = {"MAE":np.nan,"RMSE":np.nan,"MAPE%":np.nan}; used = [f"ERROR: {e}"]
    rows.append({
        "target": TARGET,
        "sNaive_RMSE": round(m_snaive["RMSE"],1), "SARIMA_RMSE": round(m_sarima["RMSE"],1), "SARIMAX_RMSE": round(m_sarimax["RMSE"],1),
        "sNaive_MAE": round(m_snaive["MAE"],1), "SARIMA_MAE": round(m_sarima["MAE"],1), "SARIMAX_MAE": round(m_sarimax["MAE"],1),
        "sNaive_MAPE%": round(m_snaive["MAPE%"],2), "SARIMA_MAPE%": round(m_sarima["MAPE%"],2), "SARIMAX_MAPE%": round(m_sarimax["MAPE%"],2),
        "SARIMAX_exog": used
    })
tbl = pd.DataFrame(rows)
print("=== ÜÇ HEDEF İÇİN TOPLU MODELLER ===")
print(tbl.to_string(index=False))


In [ ]:
# G2-EXTRA: new_comments — sNaive vs SARIMA vs SARIMAX (lag0 sabit terimler)
import pandas as pd, numpy as np
from pathlib import Path
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX

CLEAN = Path("/kaggle/working/clean")

def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

# --- verileri yükle ---
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")   # date, category, term, interest
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")        # date, new_projects, new_users, new_comments
tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])

# hedef seri
TARGET = "new_comments"
y = sc.set_index("date")[TARGET].asfreq("MS")
TEST_H = 24
y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]

# sNaive: geçen yılın aynı ayı
snaive_fc = y.shift(12).iloc[-TEST_H:]

# SARIMA düzeni
a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
               stepwise=True, suppress_warnings=True, error_action="ignore")
order, so = a.order, a.seasonal_order
sarima = SARIMAX(y_tr, order=order, seasonal_order=so,
                 enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
sarima_fc = sarima.get_forecast(steps=TEST_H).predicted_mean

# --- lag=0 sabit terimlerle exog kur (G1'e göre en güçlü lag0: scratch, blockly, bee_bot) ---
tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]

lag0_terms = [
    "platformlar_araclar__scratch",
    "platformlar_araclar__blockly",
    "platformlar_araclar__bee_bot",
]
lag0_terms = [t for t in lag0_terms if t in tw.columns]  # güvenli filtre

X = pd.DataFrame(index=tw.index)
for t in lag0_terms:
    X[f"{t}__lag0"] = tw[t].rolling(3, min_periods=1).mean()   # 3M MA, lag=0

X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]

sarimax = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
                  enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
sarimax_fc = sarimax.get_forecast(steps=TEST_H, exog=X_te).predicted_mean

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {
        "MAE": float(e.abs().mean()),
        "RMSE": float(np.sqrt((e**2).mean())),
        "MAPE%": float((e.abs() / y_true.replace(0, np.nan)).mean() * 100),
    }

print("=== G2-EXTRA: new_comments — sNaive vs SARIMA vs SARIMAX(lag0) ===")
print("Kullanılan lag0 exog:", lag0_terms)
print("sNaive:", metrics(y_te, snaive_fc))
print("SARIMA:", metrics(y_te, sarima_fc))
print("SARIMAX(lag0):", metrics(y_te, sarimax_fc))


In [ ]:
# Adım 5 / R1: Otomatik rapor üretimi (Markdown + PNG görseller)
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox


plt.rcParams["figure.dpi"] = 120

CLEAN = Path("/kaggle/working/clean")
OUT = Path("/kaggle/working/report_assets")
OUT.mkdir(parents=True, exist_ok=True)

# ---------------------------
# 0) Yüklemeler ve yardımcılar
# ---------------------------
def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

def minmax01(s):
    s = s.astype(float)
    lo, hi = s.min(), s.max()
    return (s - lo) / (hi - lo) if hi > lo else s*0 + 0.5

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {"MAE": float(e.abs().mean()),
            "RMSE": float(np.sqrt((e**2).mean())),
            "MAPE%": float((e.abs()/y_true.replace(0,np.nan)).mean()*100)}

tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
corr_sum = pd.read_csv(CLEAN / "corr_lag_summary.csv")
# opsiyonel
try:
    granger_seas = pd.read_csv(CLEAN / "granger_seasonal_summary.csv")
except FileNotFoundError:
    granger_seas = pd.DataFrame()

tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])

# Wide görünümler
tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
sw = sc.set_index("date")[["new_projects","new_users","new_comments"]].asfreq("MS").sort_index()

# ---------------------------
# 1) Grafikler: EDA
# ---------------------------
# 1A: Kategori ortalamaları (3M MA)
cat_mean = (tr.groupby(["date","category"])["interest"].mean()
              .unstack("category").sort_index().rolling(3, min_periods=1).mean())
plt.figure(figsize=(10,4))
for c in cat_mean.columns:
    plt.plot(cat_mean.index, cat_mean[c], label=c)
plt.title("Google Trends — Kategori Ortalamaları (3M MA)")
plt.xlabel("Tarih"); plt.ylabel("İlgi (0–100)"); plt.legend(loc="upper left", ncol=2, fontsize=8)
plt.tight_layout(); plt.savefig(OUT / "eda_cat_means.png"); plt.close()

# 1B: Scratch metrikleri (3M MA)
sw_ma = sw.rolling(3, min_periods=1).mean()
plt.figure(figsize=(10,4))
for c in sw_ma.columns:
    plt.plot(sw_ma.index, sw_ma[c], label=c)
plt.title("Scratch metrikleri (3M MA)")
plt.xlabel("Tarih"); plt.ylabel("Sayı"); plt.legend(loc="upper left")
plt.tight_layout(); plt.savefig(OUT / "eda_scratch_ma.png"); plt.close()

# 1C: 'scratch' trendi vs new_users (normalize 0–100, 3M MA)
x = tw["platformlar_araclar__scratch"].rolling(3, min_periods=1).mean()
y = sw["new_users"].rolling(3, min_periods=1).mean()
both = pd.concat([minmax01(x).rename("scratch_trend_norm"),
                  minmax01(y).rename("new_users_norm")], axis=1).dropna()*100
plt.figure(figsize=(10,4))
plt.plot(both.index, both["scratch_trend_norm"], label="scratch trend (norm)")
plt.plot(both.index, both["new_users_norm"], label="new_users (norm)")
plt.title("Scratch trend vs new_users (normalize 0–100, 3M MA)")
plt.xlabel("Tarih"); plt.ylabel("Normalize (0–100)"); plt.legend(loc="upper left")
plt.tight_layout(); plt.savefig(OUT / "eda_scratch_vs_users.png"); plt.close()

# 1D: Lag eğrisi (scratch vs new_users) [-12..+12]
def corr_with_lags(x, y, lags=range(-12,13), method="pearson"):
    rows=[]
    for k in lags:
        ys = y.shift(k)
        v = x.notna() & ys.notna()
        if v.sum()>=12:
            r = (x[v].corr(ys[v]) if method=="pearson" else x[v].rank().corr(ys[v].rank()))
            rows.append((k, r))
    return pd.DataFrame(rows, columns=["lag","r"])

x_ma = x.copy(); y_ma = y.copy()
dfp = corr_with_lags(x_ma, y_ma, method="pearson")
dfs = corr_with_lags(x_ma, y_ma, method="spearman")
plt.figure(figsize=(8,3.5))
plt.plot(dfp["lag"], dfp["r"], label="pearson")
plt.plot(dfs["lag"], dfs["r"], label="spearman", linestyle="--")
plt.axvline(0, color="k", alpha=.2); plt.axvline(12, color="k", alpha=.1); plt.axvline(-12, color="k", alpha=.1)
plt.title("Lag eğrisi: scratch trend ↔ new_users")
plt.xlabel("Lag (ay)  |  +: Trend önce"); plt.ylabel("r"); plt.legend()
plt.tight_layout(); plt.savefig(OUT / "lag_curve_scratch_users.png"); plt.close()

# ---------------------------
# 2) Tablolar: En güçlü korelasyonlar + Granger
# ---------------------------
top_corr = corr_sum.sort_values("r", key=lambda s: s.abs(), ascending=False).head(12)
top_corr_path = OUT / "top_corr.csv"; top_corr.to_csv(top_corr_path, index=False)

if not granger_seas.empty:
    top_granger = granger_seas.sort_values("best_p").head(10)
    top_granger_path = OUT / "top_granger.csv"; top_granger.to_csv(top_granger_path, index=False)
else:
    top_granger_path = None

# ---------------------------
# 3) Tahmin grafiği (new_users) — baseline vs exog_best
# ---------------------------
fc_tbl = pd.read_csv(CLEAN / "forecast_new_users.csv", index_col=0, parse_dates=True)
plt.figure(figsize=(10,4))
plt.plot(pd.concat([sw["new_users"].iloc[:-24], fc_tbl["actual"]]), label="gerçek")
plt.plot(fc_tbl.index, fc_tbl["baseline"], label="baseline (SARIMA)")
if fc_tbl["exog_best"].notna().any():
    plt.plot(fc_tbl.index, fc_tbl["exog_best"], label="exog_best (SARIMAX)")
plt.title("Tahmin — new_users (son 24 ay test)")
plt.xlabel("Tarih"); plt.ylabel("new_users"); plt.legend(loc="upper left")
plt.tight_layout(); plt.savefig(OUT / "forecast_new_users.png"); plt.close()

# ---------------------------
# 4) Platform karşılaştırma (lag=0 ve en iyi 0..6) — tablo CSV
# ---------------------------
platform_terms = [
    "platformlar_araclar__scratch","platformlar_araclar__tynker","platformlar_araclar__code_org",
    "platformlar_araclar__blockly","platformlar_araclar__kodable","platformlar_araclar__bee_bot",
    "platformlar_araclar__code_a_pillar","platformlar_araclar__matatalab"
]
platform_terms = [t for t in platform_terms if t in tw.columns]

def best_lag_corr(x, y, lags=range(0,7), method="pearson"):
    rows=[]
    for k in lags:
        ys = y.shift(k)
        v = x.notna() & ys.notna()
        if v.sum()>=12:
            r = (x[v].corr(ys[v]) if method=="pearson" else x[v].rank().corr(ys[v].rank()))
            rows.append((k, r, int(v.sum())))
    if not rows: 
        return None
    best = max(rows, key=lambda t: abs(t[1]))
    return {"lag":best[0], "r":best[1], "n":best[2]}

tw_ma = tw.rolling(3, min_periods=1).mean(); sw_ma = sw.rolling(3, min_periods=1).mean()
out=[]
for term in platform_terms:
    x = tw_ma[term]
    for m in sw_ma.columns:
        y = sw_ma[m]
        v = x.notna() & y.notna(); n0=int(v.sum())
        r0p = x[v].corr(y[v]); r0s = x[v].rank().corr(y[v].rank())
        bp = best_lag_corr(x,y,method="pearson"); bs = best_lag_corr(x,y,method="spearman")
        out.append({
            "term": term.split("__",1)[1],
            "metric": m,
            "r0_pearson": round(r0p,3), "r0_spearman": round(r0s,3), "n0": n0,
            "best_lag_p": (None if not bp else bp["lag"]),
            "best_r_p":   (None if not bp else round(bp["r"],3)),
            "best_lag_s": (None if not bs else bs["lag"]),
            "best_r_s":   (None if not bs else round(bs["r"],3)),
            "n_best":     (None if not bp else bp["n"])
        })
platform_compare = pd.DataFrame(out).sort_values(["metric","best_r_s"], ascending=[True,False])
platform_compare_path = OUT / "platform_compare.csv"
platform_compare.to_csv(platform_compare_path, index=False)

# ---------------------------
# 5) Üç hedef için toplu model kıyası (sNaive vs SARIMA vs SARIMAX)
# ---------------------------
def build_exog_for_target(target, k=3):
    c = (corr_sum.query("method=='pearson' and metric==@target")
                 .query("best_lag>=0 and best_lag<=6")
                 .sort_values("r", key=lambda s: s.abs(), ascending=False)
                 .drop_duplicates("term").head(k))[["term","best_lag"]].values.tolist()
    X = pd.DataFrame(index=tw.index)
    used=[]
    for t,lag in c:
        col = f"{t}__lag{int(lag)}"
        X[col] = tw[t].rolling(3, min_periods=1).mean().shift(int(lag))
        used.append(col)
    return X, used

rows=[]
for TARGET in ["new_users","new_projects","new_comments"]:
    y = sw[TARGET]
    TEST_H=24
    y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]
    snaive_fc = y.shift(12).iloc[-TEST_H:]
    m_snaive = metrics(y_te, snaive_fc)

    a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
                   stepwise=True, suppress_warnings=True, error_action="ignore")
    order, so = a.order, a.seasonal_order
    sarima = SARIMAX(y_tr, order=order, seasonal_order=so,
                     enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    base_fc = sarima.get_forecast(steps=TEST_H).predicted_mean
    m_sarima = metrics(y_te, base_fc)

    X, used = build_exog_for_target(TARGET, k=3)
    X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]
    try:
        sarimax = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
                          enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
        ex_fc = sarimax.get_forecast(steps=TEST_H, exog=X_te).predicted_mean
        m_sarimax = metrics(y_te, ex_fc)
    except Exception as e:
        m_sarimax = {"MAE":np.nan,"RMSE":np.nan,"MAPE%":np.nan}
        used = [f"ERROR: {e}"]
    rows.append({
        "target": TARGET,
        "sNaive_RMSE": round(m_snaive["RMSE"],1), "SARIMA_RMSE": round(m_sarima["RMSE"],1), "SARIMAX_RMSE": round(m_sarimax["RMSE"],1),
        "sNaive_MAE": round(m_snaive["MAE"],1), "SARIMA_MAE": round(m_sarima["MAE"],1), "SARIMAX_MAE": round(m_sarimax["MAE"],1),
        "sNaive_MAPE%": round(m_snaive["MAPE%"],2), "SARIMA_MAPE%": round(m_sarima["MAPE%"],2), "SARIMAX_MAPE%": round(m_sarimax["MAPE%"],2),
        "SARIMAX_exog": used
    })
model_tbl = pd.DataFrame(rows)
model_tbl_path = OUT / "models_all_targets.csv"
model_tbl.to_csv(model_tbl_path, index=False)

# ---------------------------
# 6) Artık tanı (new_users, en iyi 3 exog)
# ---------------------------
cand = (corr_sum.query("method=='pearson' and metric=='new_users'")
             .query("best_lag>=0 and best_lag<=6")
             .sort_values("r", key=lambda s: s.abs(), ascending=False)
             .drop_duplicates("term").head(3))[["term","best_lag"]].values.tolist()
X = pd.DataFrame(index=tw.index)
for t,lag in cand:
    X[f"{t}__lag{int(lag)}"] = tw[t].rolling(3, min_periods=1).mean().shift(int(lag))

y = sw["new_users"]; TEST_H=24
y_tr, y_te = y.iloc[:-TEST_H], y.iloc[-TEST_H:]
X_tr, X_te = X.loc[y_tr.index], X.loc[y_te.index]
a = auto_arima(y_tr, seasonal=True, m=12, information_criterion="aic",
               stepwise=True, suppress_warnings=True, error_action="ignore")
order, so = a.order, a.seasonal_order
m = SARIMAX(y_tr, exog=X_tr, order=order, seasonal_order=so,
            enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
resid = pd.Series(m.filter_results.standardized_forecasts_error[0], index=y_tr.index).dropna()
lb = acorr_ljungbox(resid, lags=[12,24], return_df=True).round(4)
lb_path = OUT / "ljungbox_new_users.csv"; lb.to_csv(lb_path, index=False)

# ---------------------------
# 7) Markdown dosyasını yaz
# ---------------------------
md = f"""# Google Trends × Scratch Analizi — Rapor

**Veri setleri:** Google Trends (kategoriler/terimler) ve Scratch (new_users, new_projects, new_comments).  
**Amaç:** Trends'teki ilgi ile Scratch kullanım metrikleri arasındaki ilişkiyi incelemek; tahminlerde exog katkısını test etmek.

---

## 1) Betimsel Analiz (EDA)
- Kategorilerin yıllar içindeki ortalamaları (3M MA):  
  ![](report_assets/eda_cat_means.png)
- Scratch metrikleri (3M MA):  
  ![](report_assets/eda_scratch_ma.png)
- 'scratch' arama ilgisi × `new_users` (normalize 0–100):  
  ![](report_assets/eda_scratch_vs_users.png)

**Not:** Güçlü 12-ay sezonsallığı gözleniyor.

---

## 2) İlişki Analizi
- Lag eğrisi ('scratch' ↔ `new_users`):  
  ![](report_assets/lag_curve_scratch_users.png)

- **En güçlü 12 ilişki (lag taraması, Pearson/Spearman):**  
  (dosya: `report_assets/top_corr.csv`)

- **Mevsimsel farkla Granger (özet):**  
  {('dosya: report_assets/top_granger.csv' if top_granger_path else 'uygun anlamlı sonuç yok veya dosya üretilmedi')}

---

## 3) Platform Karşılaştırması
Tablo: `report_assets/platform_compare.csv`  
(Lag=0 korelasyonları ve **0..6 ay** içinde en yüksek korelasyon lagı.)

---

## 4) Tahmin (Son 24 ay test)
- `new_users` tahmin grafiği:  
  ![](report_assets/forecast_new_users.png)

- **Üç hedef için kıyas** (sNaive vs SARIMA vs SARIMAX):  
  Tablo: `report_assets/models_all_targets.csv`

**Özet:** SARIMAX, SARIMA'ya göre belirgin iyileşme sağlasa da **sNaive** (y_{ '{t-12}' }) çoğu hedefte en düşük hatayı verir.

---

## 5) Artık Tanı (new_users, SARIMAX)
- Ljung–Box p-değerleri (12, 24): dosya `report_assets/ljungbox_new_users.csv`  
  (Yüksek p-değeri → artıklar beyaz gürültüye yakın.)

---

## 6) Sonuçlar ve Öneriler
- **İlişki:** Yüksek korelasyonlar büyük ölçüde **eşzamanlı** ve **yıllık sezonsallık** kaynaklıdır. Kısa vadeli lider sinyal adayları: `coding_in_school (≈+4 ay)` ve sınırlı olarak `matatalab (≈+6 ay)`.
- **Tahmin:** Exog'lar (Trends) SARIMA'yı iyileştirir; fakat sNaive çoğu hedefte daha iyi kalır.
- **Uygulama:** Üretimde **sNaive**'i baseline yapın; üstüne **residual-correction** (sNaive artıkları ≈ exog ile modelleme) deneyin. Operasyonel ileri tahmin için **lag≥1** exog'lar kullanılmalıdır.
"""

(Path("/kaggle/working") / "report.md").write_text(md, encoding="utf-8")

print("=== ADIM 5 / R1 TAMAMLANDI ===")
print("Rapor:", "/kaggle/working/report.md")
print("Varlık klasörü:", str(OUT))
print("Ek tablolar:")
print("-", top_corr_path)
if top_granger_path: print("-", top_granger_path)
print("-", platform_compare_path)
print("-", model_tbl_path)
print("-", lb_path)


In [ ]:
# R2: STL decomposition (period=12) görselleri
import pandas as pd, matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from pathlib import Path

plt.rcParams["figure.dpi"] = 120

CLEAN = Path("/kaggle/working/clean")
OUT   = Path("/kaggle/working/report_assets")
OUT.mkdir(parents=True, exist_ok=True)

sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
sc["date"] = pd.to_datetime(sc["date"]).dt.to_period("M").dt.to_timestamp(how="start")
sw = sc.set_index("date")[["new_users","new_projects","new_comments"]].asfreq("MS")

for col in sw.columns:
    y = sw[col].dropna()
    stl = STL(y, period=12, robust=True).fit()
    fig = stl.plot()
    fig.set_size_inches(10,6)
    fig.suptitle(f"STL Decomposition — {col} (period=12)", fontsize=12, y=1.02)
    (OUT / f"stl_{col}.png").unlink(missing_ok=True)
    fig.savefig(OUT / f"stl_{col}.png", bbox_inches="tight")
    plt.close(fig)

print("STL görselleri kaydedildi:", [f"stl_{c}.png" for c in sw.columns])


In [ ]:
# R3: sNaive + Residual-Correction (lag>=1 exog) — üç hedef için
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

plt.rcParams["figure.dpi"] = 120

CLEAN = Path("/kaggle/working/clean")
OUT   = Path("/kaggle/working/report_assets")
OUT.mkdir(parents=True, exist_ok=True)

def mstart(s):
    s = pd.to_datetime(s, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp(how="start")

def metrics(y_true, y_pred):
    e = (y_pred - y_true).astype(float)
    return {"MAE": float(e.abs().mean()),
            "RMSE": float(np.sqrt((e**2).mean())),
            "MAPE%": float((e.abs()/y_true.replace(0,np.nan)).mean()*100)}

# veriler
tr = pd.read_parquet(CLEAN / "google_trends_long.parquet")
sc = pd.read_parquet(CLEAN / "scratch_clean.parquet")
corr = pd.read_csv(CLEAN / "corr_lag_summary.csv")

tr["date"] = mstart(tr["date"])
sc["date"] = mstart(sc["date"])

tw = tr.pivot(index="date", columns=["category","term"], values="interest").asfreq("MS").sort_index()
tw.columns = [f"{a}__{b}" for a,b in tw.columns]
sw = sc.set_index("date")[["new_users","new_projects","new_comments"]].asfreq("MS").sort_index()

TEST_H = 24
rows=[]
plots_done=False

for TARGET in sw.columns:
    y = sw[TARGET]
    y_train, y_test = y.iloc[:-TEST_H], y.iloc[-TEST_H:]

    # sNaive baseline
    snaive_train = y_train.shift(12)
    snaive_test  = y.shift(12).iloc[-TEST_H:]

    # Residual seri (train dönemi için)
    resid_train = (y_train - snaive_train).dropna()

    # Exog seçimi: lag in [1..6], en güçlü 3 terim (Pearson)
    sel = (corr.query("method=='pearson' and metric==@TARGET")
                .query("best_lag>=1 and best_lag<=6")
                .sort_values("r", key=lambda s: s.abs(), ascending=False)
                .drop_duplicates("term").head(3))
    exog_specs = [(t, int(l)) for t,l in sel[["term","best_lag"]].itertuples(index=False, name=None)]

    # fallback: hiç bulunamazsa scratch lag1 ekle
    if not exog_specs and "platformlar_araclar__scratch" in tw.columns:
        exog_specs = [("platformlar_araclar__scratch", 1)]

    # Exog matrisi
    X_all = pd.DataFrame(index=tw.index)
    for t,lag in exog_specs:
        X_all[f"{t}__lag{lag}"] = tw[t].rolling(3, min_periods=1).mean().shift(lag)

    # Train/test eşitleme (resid_train indeksine göre)
    X_tr = X_all.loc[resid_train.index]
    X_te = X_all.loc[y_test.index]

    # Pipeline: scale + ridge
    pipe = Pipeline([("scaler", StandardScaler(with_mean=True, with_std=True)),
                     ("ridge", Ridge(alpha=1.0, random_state=0))])
    # fit sadece ortak NaN olmayanlarda
    v = X_tr.notna().all(axis=1)
    pipe.fit(X_tr[v], resid_train[v])

    # Tahmin: test artıkları
    resid_pred = pd.Series(pipe.predict(X_te), index=X_te.index)
    # Nihai tahmin = sNaive + tahmin edilen artık
    final_fc = snaive_test.add(resid_pred, fill_value=0.0)

    # Metrikler
    m_snaive = metrics(y_test, snaive_test)
    m_final  = metrics(y_test, final_fc)

    rows.append({
        "target": TARGET,
        "features": [f"{t}__lag{l}" for t,l in exog_specs],
        "sNaive_MAE": round(m_snaive["MAE"],1), "sNaive_RMSE": round(m_snaive["RMSE"],1), "sNaive_MAPE%": round(m_snaive["MAPE%"],2),
        "ResCorr_MAE": round(m_final["MAE"],1), "ResCorr_RMSE": round(m_final["RMSE"],1), "ResCorr_MAPE%": round(m_final["MAPE%"],2),
        "improve_RMSE_%": round((m_snaive["RMSE"]-m_final["RMSE"])/m_snaive["RMSE"]*100,2)
    })

    # Kaydet: tahmin CSV
    out_tbl = pd.DataFrame({"actual": y_test, "sNaive": snaive_test, "residual_fc": final_fc})
    out_tbl.to_csv(OUT / f"residual_forecast_{TARGET}.csv")

    # Sadece new_users için grafik çiz (rapora koyacağız)
    if TARGET=="new_users":
        plt.figure(figsize=(10,4))
        plt.plot(pd.concat([y_train, y_test]).index, pd.concat([y_train, y_test]), label="gerçek")
        plt.plot(y_test.index, snaive_test, label="sNaive")
        plt.plot(y_test.index, final_fc, label="sNaive + residual-correction")
        plt.title("Residual-Correction Forecast — new_users (son 24 ay)")
        plt.xlabel("Tarih"); plt.ylabel("new_users"); plt.legend(loc="upper left")
        plt.tight_layout(); plt.savefig(OUT / "residcorr_forecast_new_users.png"); plt.close()

res_tbl = pd.DataFrame(rows)
res_tbl.to_csv(OUT / "residual_correction_summary.csv", index=False)

print("Artık-düzeltme özet tablosu:", OUT / "residual_correction_summary.csv")
print(res_tbl.to_string(index=False))


In [ ]:
# R4: report.md'ye ek bölümleri yaz (STL + Residual-Correction)
from pathlib import Path

base = Path("/kaggle/working")
OUT  = base / "report_assets"

append_md = f"""

---

## 7) STL Parçalama (Trend/Seasonal/Resid)
Aşağıdaki görseller 12 aylık mevsimselliği ve uzun dönem eğilimi açıkça göstermektedir:
- ![](report_assets/stl_new_users.png)
- ![](report_assets/stl_new_projects.png)
- ![](report_assets/stl_new_comments.png)

## 8) sNaive + Artık-Düzeltme (operasyonel: lag≥1)
- Özet tablo: `report_assets/residual_correction_summary.csv`
- `new_users` görseli:  
  ![](report_assets/residcorr_forecast_new_users.png)

**Yorum:** Artık-düzeltme yaklaşımı, sNaive'e kıyasla seçilen hedef ve exog kombinasyonlarına göre { '{' }iyileşme veya benzer performans{ '}' } gösterebilir. Bu yöntem, sezonsallığın baskın olduğu serilerde “sNaive”in güçlü performansını bozmadan, ek sinyal oldukça hata azaltımı sağlar.
"""

p = base / "report.md"
with p.open("a", encoding="utf-8") as f:
    f.write(append_md)

print("Rapor güncellendi:", p)


In [ ]:
# R5: Rapor + varlıkları paketle (ZIP)
import zipfile
from pathlib import Path

base = Path("/kaggle/working")
zip_path = base / "report_bundle.zip"

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    z.write(base / "report.md", arcname="report.md")
    for p in (base / "report_assets").rglob("*"):
        if p.is_file():
            z.write(p, arcname=f"report_assets/{p.name}")

print("Hazır ZIP:", zip_path)


In [ ]:
# PC-SUMMARY: platform_compare.csv -> 3 özet tablo
import pandas as pd, numpy as np
from pathlib import Path

ASSETS = Path("/kaggle/working/report_assets")
df = pd.read_csv(ASSETS / "platform_compare.csv")

# Sayısal kolonları garanti altına al
num_cols = ["r0_pearson","r0_spearman","best_r_p","best_r_s","n0","n_best","best_lag_p","best_lag_s"]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

TOPN = 5  # her metrik için kaç terimi istiyorsun? (istersen 10 yap)

# --- Özet A: Lag=0 (Spearman) en iyi TOPN — her metrik için
lag0 = (df.sort_values(["metric","r0_spearman"], ascending=[True,False])
          .groupby("metric").head(TOPN).copy())
lag0["rank"] = lag0.groupby("metric")["r0_spearman"].rank(ascending=False, method="first").astype(int)
lag0_out = lag0[["rank","metric","term","r0_spearman","r0_pearson","n0"]]\
            .sort_values(["metric","rank"])

# --- Özet B: En iyi lag (0..6) — Spearman — her metrik için TOPN
best = (df.sort_values(["metric","best_r_s"], ascending=[True,False])
          .groupby("metric").head(TOPN).copy())
best["rank"] = best.groupby("metric")["best_r_s"].rank(ascending=False, method="first").astype(int)
best_out = best[["rank","metric","term","best_lag_s","best_r_s","n_best"]]\
            .sort_values(["metric","rank"])

# --- Özet C: Genel görünüm — terim bazında
overall = (df.groupby("term")
             .agg(metrics_covered=("metric","nunique"),
                  r0_s_mean=("r0_spearman","mean"),
                  r0_s_min=("r0_spearman","min"),
                  r0_s_max=("r0_spearman","max"),
                  best_s_mean=("best_r_s","mean"),
                  lead_count=("best_lag_s", lambda s: np.sum(pd.to_numeric(s, errors="coerce") > 0)))
             .reset_index()
             .sort_values("r0_s_mean", ascending=False))

# Kaydet
lag0_path   = ASSETS / "platform_compare_top_lag0.csv"
best_path   = ASSETS / "platform_compare_top_bestlag.csv"
overall_path= ASSETS / "platform_compare_overall.csv"

lag0_out.to_csv(lag0_path, index=False)
best_out.to_csv(best_path, index=False)
overall.to_csv(overall_path, index=False)

print("Kaydedildi:")
print("-", lag0_path)
print("-", best_path)
print("-", overall_path)

print("\nÖrnek — Lag=0 TOPN (ilk satırlar):")
print(lag0_out.head(10).to_string(index=False))

print("\nÖrnek — En iyi lag TOPN (ilk satırlar):")
print(best_out.head(10).to_string(index=False))

print("\nÖrnek — Genel (ilk satırlar):")
print(overall.head(10).to_string(index=False))


In [ ]:
# TOP GRANGER TABLOSU — Kaggle'da güzel görünüm + HTML çıktısı
import pandas as pd
from pathlib import Path

path = Path("/kaggle/working/report_assets/top_granger.csv")
df = pd.read_csv(path)

# (Opsiyonel) En düşük p-değerine göre sırala ve ilgilendiğimiz kolonları öne al
preferred_cols = [c for c in ["term","metric","best_lag","best_p","n","sig(p<0.05)","x_transform","y_transform"] if c in df.columns]
if preferred_cols:
    df = df[preferred_cols]
df = df.sort_values("best_p", na_position="last").reset_index(drop=True)

# Ekranda daha iyi görünmesi için bazı ayarlar
pd.set_option("display.max_rows", None)       # tüm satırları göster
pd.set_option("display.max_colwidth", None)   # uzun metinler kırpılmasın

# p<0.05 hücrelerini yeşil renkle vurgula
def highlight_sig(s):
    return ['background-color: #d4edda' if (pd.notnull(v) and isinstance(v,(int,float)) and v < 0.05) else '' for v in s]

styled = (df.style
    .format({"best_p": "{:.4f}"})
    .apply(highlight_sig, subset=["best_p"])
    .set_caption("Mevsimsel farkla Granger — en düşük p-değerleri (p<0.05 yeşil)")
    .hide(axis="index")
)

# 1) Notebokta tabloyu göster (buradan screenshot alabilirsin)
styled

# 2) Aynı stilli görünümü HTML olarak da kaydet (dosyadan da ekran görüntüsü alabilirsin)
html_out = Path("/kaggle/working/report_assets/top_granger_pretty.html")
html_out.write_text(styled.to_html(), encoding="utf-8")
print("Kaydedildi:", html_out)


In [ ]:
# MODELS ALL TARGETS — Kaggle'da güzel tablo + HTML çıktısı
import pandas as pd
from pathlib import Path
import numpy as np

path = Path("/kaggle/working/report_assets/models_all_targets.csv")
df = pd.read_csv(path)

# Kolon gruplarını otomatik bul (dosya yapısına dayanarak)
rmse_cols = [c for c in df.columns if c.endswith("_RMSE")]
mae_cols  = [c for c in df.columns if c.endswith("_MAE")]
mape_cols = [c for c in df.columns if c.endswith("MAPE%")]
order = (["target"] + rmse_cols + mae_cols + mape_cols + ([ "SARIMAX_exog" ] if "SARIMAX_exog" in df.columns else []))
df = df[order]

# Görsel ayarlar
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# Biçimlendirme yardımcıları
fmt_nums = {c: "{:,.0f}" for c in (rmse_cols + mae_cols)}
fmt_nums.update({c: "{:.2f}%" for c in mape_cols})

# NaN'ları "—" göster (özellikle SARIMAX NAN olabilir)
df_display = df.copy()
for c in rmse_cols + mae_cols + mape_cols:
    df_display[c] = pd.to_numeric(df_display[c], errors="coerce")

styled = (
    df_display.style
      .format(fmt_nums, na_rep="—")
      # En düşük hataları yeşil vurgula (satır bazında)
      .highlight_min(subset=rmse_cols, color="#d4edda", axis=1)
      .highlight_min(subset=mae_cols,  color="#d4edda", axis=1)
      .highlight_min(subset=mape_cols, color="#d4edda", axis=1)
      .set_caption("Üç Hedef İçin Model Karşılaştırması — En düşük hata yeşil")
      .hide(axis="index")
)

# 1) Notebokta göster — buradan ekran görüntüsü alabilirsin
styled

# 2) Aynı stilli görünümü HTML olarak kaydet
html_out = Path("/kaggle/working/report_assets/models_all_targets_pretty.html")
html_out.write_text(styled.to_html(), encoding="utf-8")
print("Kaydedildi:", html_out)


In [ ]:
# LJUNG–BOX TABLOSU — Kaggle'da şık görünüm + HTML çıktısı
import pandas as pd
from pathlib import Path

path = Path("/kaggle/working/report_assets/ljungbox_new_users.csv")
df = pd.read_csv(path)

# Bazı kayıtlarda 'lag' kolonu CSV'ye yazılmaz; yoksa ekleyelim (12 ve 24 bekleniyor)
if "lag" not in df.columns:
    default_lags = [12, 24]
    if len(df) == len(default_lags):
        df.insert(0, "lag", default_lags)
    else:
        # emniyetli fallback: 1..n
        df.insert(0, "lag", range(1, len(df) + 1))

# Sayısal türleri garantiye al
for c in ["lag", "lb_stat", "lb_pvalue"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# Görsel ayarlar
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

# p>=0.05 (otokorelasyon YOK) yeşil; p<0.05 kırmızı
def color_pvals(col):
    out = []
    for v in col:
        if pd.isna(v):
            out.append("")
        elif v >= 0.05:
            out.append("background-color: #d4edda")  # yeşil (iyi)
        else:
            out.append("background-color: #f8d7da")  # kırmızı (kötü)
    return out

styled = (
    df.style
      .format({"lb_stat": "{:.4f}", "lb_pvalue": "{:.4f}"})
      .apply(color_pvals, subset=["lb_pvalue"])
      .set_caption("Ljung–Box Test (new_users) — p≥0.05: otokorelasyon gözlenmedi")
      .hide(axis="index")
)

# 1) Notebokta tabloyu göster — buradan ekran görüntüsü alabilirsin
styled

# 2) Aynı görünümü HTML olarak kaydet
html_out = Path("/kaggle/working/report_assets/ljungbox_new_users_pretty.html")
html_out.write_text(styled.to_html(), encoding="utf-8")
print("Kaydedildi:", html_out)


In [ ]:
# RESIDUAL-CORRECTION ÖZETİ — Kaggle'da şık tablo + HTML çıktısı
import pandas as pd
from pathlib import Path
import ast

path = Path("/kaggle/working/report_assets/residual_correction_summary.csv")
df = pd.read_csv(path)

# (Opsiyonel) features sütununu okunur hale getir (liste gibi geldiyse)
if "features" in df.columns:
    def prettify_feats(x):
        try:
            v = ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith(("[" , "(")) else x
            if isinstance(v, (list, tuple)):
                return ", ".join(map(str, v))
            return str(v)
        except Exception:
            return str(x)
    df["features"] = df["features"].apply(prettify_feats)

# Sayısal kolonları biçimlendirme için güvenceye al
num_cols = [c for c in df.columns if any(k in c for k in ["MAE","RMSE","MAPE","improve_RMSE_%"])]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Biçimlendirme şablonları
fmt = {c: "{:,.0f}" for c in df.columns if c.endswith(("MAE","RMSE"))}
fmt.update({c: "{:.2f}%" for c in df.columns if c.endswith(("MAPE%","improve_RMSE_%"))})

# İyileşme yüzdesini renklendir (pozitif yeşil, negatif kırmızı)
def color_improve(s):
    out=[]
    for v in s:
        if pd.isna(v):
            out.append("")
        elif v >= 0:
            out.append("background-color: #d4edda")  # yeşil
        else:
            out.append("background-color: #f8d7da")  # kırmızı
    return out

# Satır içi en düşük hata (ResCorr vs sNaive) vurgusu için sütun listeleri
rmse_cols = [c for c in df.columns if c.endswith("_RMSE")]
mae_cols  = [c for c in df.columns if c.endswith("_MAE")]
mape_cols = [c for c in df.columns if c.endswith("MAPE%")]

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

styled = (
    df.style
      .format(fmt, na_rep="—")
      .apply(color_improve, subset=["improve_RMSE_%"])            # iyileşme rengi
      .highlight_min(subset=rmse_cols, color="#e2f0d9", axis=1)   # en düşük RMSE açık yeşil
      .highlight_min(subset=mae_cols,  color="#e2f0d9", axis=1)   # en düşük MAE açık yeşil
      .highlight_min(subset=mape_cols, color="#e2f0d9", axis=1)   # en düşük MAPE açık yeşil
      .set_caption("sNaive + Residual-Correction Özeti — improve_RMSE_% pozitifse (yeşil) daha iyi")
      .hide(axis="index")
)

# 1) Notebokta tabloyu göster — buradan SS alabilirsin
styled

# 2) Aynı stilli görünümü HTML olarak kaydet
html_out = Path("/kaggle/working/report_assets/residual_correction_summary_pretty.html")
html_out.write_text(styled.to_html(), encoding="utf-8")
print("Kaydedildi:", html_out)


In [ ]:
# APPX-ALL-PRETTY — 3 appendix tablosunu şık göster + HTML kaydet
import pandas as pd
from pathlib import Path

ASSETS = Path("/kaggle/working/report_assets")

def pretty_platform(csv_name, caption, sort_col, html_name):
    df = pd.read_csv(ASSETS / csv_name)
    # sayısalları güvenceye al
    num_like = ["r0_pearson","r0_spearman","best_r_p","best_r_s","n0","n_best","best_lag_p","best_lag_s"]
    for c in df.columns:
        if c in num_like:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    # sıralama (yüksek korelasyon üstte)
    if sort_col in df.columns:
        df = df.sort_values(sort_col, ascending=False)
    # stil
    grad_cols = [c for c in df.columns if c.startswith(("r0_","best_r_"))]
    styled = (
        df.style
          .format({c:"{:.3f}" for c in grad_cols})
          .format({c:"{:.0f}" for c in df.columns if c in ["n0","n_best","best_lag_p","best_lag_s"]})
          .background_gradient(subset=grad_cols, cmap="Greens")
          .set_caption(caption)
          .hide(axis="index")
    )
    html_out = ASSETS / html_name
    html_out.write_text(styled.to_html(), encoding="utf-8")
    print("Kaydedildi:", html_out)
    return styled

# A) Platform Compare — Lag=0 (Spearman)
styled_lag0 = pretty_platform(
    "platform_compare_top_lag0.csv",
    "Platform Compare — Lag=0 (Spearman, en iyi TOPN)",
    "r0_spearman",
    "platform_compare_top_lag0_pretty.html"
)
styled_lag0  # notebokta gösterilir (buradan SS al)

# B) Platform Compare — En iyi lag 0..6 (Spearman)
styled_best = pretty_platform(
    "platform_compare_top_bestlag.csv",
    "Platform Compare — En iyi lag 0..6 (Spearman, en iyi TOPN)",
    "best_r_s",
    "platform_compare_top_bestlag_pretty.html"
)
styled_best  # notebokta gösterilir (buradan SS al)

# C) Residual-Correction Özeti (daha önce üretmediysen yeniden üretelim)
def pretty_rescorr():
    df = pd.read_csv(ASSETS / "residual_correction_summary.csv")
    # sayısallar
    for c in df.columns:
        if any(k in c for k in ["MAE","RMSE","MAPE","improve_RMSE_%"]):
            df[c] = pd.to_numeric(df[c], errors="coerce")
    fmt = {c:"{:,.0f}" for c in df.columns if c.endswith(("MAE","RMSE"))}
    fmt.update({c:"{:.2f}%" for c in df.columns if c.endswith(("MAPE%","improve_RMSE_%"))})
    def color_improve(col):
        return ["background-color: #d4edda" if (pd.notnull(v) and v>=0) else ("background-color: #f8d7da" if pd.notnull(v) else "") for v in col]
    rmse_cols = [c for c in df.columns if c.endswith("_RMSE")]
    mae_cols  = [c for c in df.columns if c.endswith("_MAE")]
    mape_cols = [c for c in df.columns if c.endswith("MAPE%")]
    styled = (
        df.style
          .format(fmt, na_rep="—")
          .apply(color_improve, subset=["improve_RMSE_%"])
          .highlight_min(subset=rmse_cols, color="#e2f0d9", axis=1)
          .highlight_min(subset=mae_cols,  color="#e2f0d9", axis=1)
          .highlight_min(subset=mape_cols, color="#e2f0d9", axis=1)
          .set_caption("sNaive + Residual-Correction Özeti — improve_RMSE_% pozitifse (yeşil) daha iyi")
          .hide(axis="index")
    )
    html_out = ASSETS / "residual_correction_summary_pretty.html"
    html_out.write_text(styled.to_html(), encoding="utf-8")
    print("Kaydedildi:", html_out)
    return styled

styled_res = pretty_rescorr()
styled_res  # notebokta gösterilir (buradan SS al)


In [ ]:
# TÜM ÇIKTILARI ZIPLE: /kaggle/working -> /kaggle/working/all_outputs.zip
import os, zipfile
from pathlib import Path

BASE = Path("/kaggle/working")
ZIP_PATH = BASE / "all_outputs.zip"

EXCLUDE_DIRS = {".ipynb_checkpoints", "__pycache__"}
EXCLUDE_EXTS = {".zip"}  # mevcut zip'leri dahil etme

def should_exclude(p: Path) -> bool:
    if any(part in EXCLUDE_DIRS for part in p.parts):
        return True
    if p.suffix.lower() in EXCLUDE_EXTS:
        return True
    return False

with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for root, dirs, files in os.walk(BASE):
        # hariç klasörleri yürüyüşten çıkar
        dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]
        for fname in files:
            fpath = Path(root) / fname
            if should_exclude(fpath) or fpath.samefile(ZIP_PATH):
                continue
            # arşiv içinde göreli yol kullan
            arcname = fpath.relative_to(BASE)
            z.write(fpath, arcname=str(arcname))

print("Hazır ZIP:", ZIP_PATH)
print("Toplam dosya:", len(zipfile.ZipFile(ZIP_PATH).namelist()))
